# Feature Engineering en SQL

A continuación, veremos cómo calcular diferentes variables para el feature engineering utilizando SQL.


In [41]:
%pip install duckdb
%pip install jupysql
%pip install duckdb-engine

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

#%sql duckdb:///:memory:
#%sql duckdb:///dm_eyf.db
%sql duckdb:///../../../db/dm_eyf.db

In [2]:
dataset_path = 'C:/Users/Federico/Desktop/Maestria Data mining/DM EyF/datasets/'
dataset_file = 'competencia_01.csv'

In [3]:
%%sql
create or replace table competencia_01 as
select
    *
from read_csv_auto("../../../datasets/competencia_01.csv")

,Success


In [4]:
df=pd.read_csv(dataset_path + dataset_file)

In [5]:
campos=df.columns.to_list()
print(campos[-1])
campos.pop(-1) #elimino clase ternaria
print(campos[-1])

clase_ternaria
Visa_mpagominimo


Ahora creo una tabla con las nuevas columnas, tengo que hacer la macro por que estoy dividiendo por 0.. 

In [6]:
%%sql
CREATE OR REPLACE MACRO suma_sin_null(a, b) AS ifnull(a, 0) + ifnull(b, 0)

,Success


In [7]:
%%sql
CREATE OR REPLACE MACRO div_sin_cero(a, b) as
    case
        when ifnull(b, 0) = 0 THEN NULL
        else ifnull(a, 0) / ifnull(b, 1)
    end

,Success


In [8]:
# Separar las columnas por tipo
master = [col for col in campos if 'Master' in col]
visa = [col for col in campos if 'Visa' in col]

sumas = []

for m_col, v_col in zip(master, visa):
    # Crear la suma usando la función 'suma_sin_null'
    suma = f"suma_sin_null({m_col}, {v_col}) as {m_col.replace('Master_', '')}_total"
    sumas.append(suma)

consulta_sql = f"""
select 
    numero_de_cliente,
    {', '.join(sumas)}
from competencia_01;
"""

print(consulta_sql)


select 
    numero_de_cliente,
    suma_sin_null(Master_delinquency, Visa_delinquency) as delinquency_total, suma_sin_null(Master_status, Visa_status) as status_total, suma_sin_null(Master_mfinanciacion_limite, Visa_mfinanciacion_limite) as mfinanciacion_limite_total, suma_sin_null(Master_Fvencimiento, Visa_Fvencimiento) as Fvencimiento_total, suma_sin_null(Master_Finiciomora, Visa_Finiciomora) as Finiciomora_total, suma_sin_null(Master_msaldototal, Visa_msaldototal) as msaldototal_total, suma_sin_null(Master_msaldopesos, Visa_msaldopesos) as msaldopesos_total, suma_sin_null(Master_msaldodolares, Visa_msaldodolares) as msaldodolares_total, suma_sin_null(Master_mconsumospesos, Visa_mconsumospesos) as mconsumospesos_total, suma_sin_null(Master_mconsumosdolares, Visa_mconsumosdolares) as mconsumosdolares_total, suma_sin_null(Master_mlimitecompra, Visa_mlimitecompra) as mlimitecompra_total, suma_sin_null(Master_madelantopesos, Visa_madelantopesos) as madelantopesos_total, suma_sin_null(Ma

In [9]:
%%sql
create or replace table competencia_01_suma_division as
select
    *
    ,ifnull(cliente_antiguedad, 0) as cliente_antiguedad_no_null,
    ifnull(mrentabilidad, 0) as mrentabilidad_no_null,
    ifnull(mrentabilidad_annual, 0) as mrentabilidad_annual_no_null,
    case
        when cliente_antiguedad_no_null > 11 then 
            cast(mrentabilidad_no_null > (mrentabilidad_annual_no_null / 12) as integer)
        else 
            cast(mrentabilidad_no_null > (mrentabilidad_annual_no_null / cliente_antiguedad_no_null) as integer)
    end as es_rentable,
    suma_sin_null(Master_delinquency, Visa_delinquency) as delinquency_total,
    suma_sin_null(Master_status, Visa_status) as status_total,
    suma_sin_null(Master_mfinanciacion_limite, Visa_mfinanciacion_limite) as mfinanciacion_limite_total,
    suma_sin_null(Master_Fvencimiento, Visa_Fvencimiento) as Fvencimiento_total,
    suma_sin_null(Master_Finiciomora, Visa_Finiciomora) as Finiciomora_total,
    suma_sin_null(Master_msaldototal, Visa_msaldototal) as msaldototal_total,
    suma_sin_null(Master_msaldopesos, Visa_msaldopesos) as msaldopesos_total,
    suma_sin_null(Master_msaldodolares, Visa_msaldodolares) as msaldodolares_total,
    suma_sin_null(Master_mconsumospesos, Visa_mconsumospesos) as mconsumospesos_total,
    suma_sin_null(Master_mconsumosdolares, Visa_mconsumosdolares) as mconsumosdolares_total,
    suma_sin_null(Master_mlimitecompra, Visa_mlimitecompra) as mlimitecompra_total,
    suma_sin_null(Master_madelantopesos, Visa_madelantopesos) as madelantopesos_total,
    suma_sin_null(Master_madelantodolares, Visa_madelantodolares) as madelantodolares_total,
    suma_sin_null(Master_fultimo_cierre, Visa_fultimo_cierre) as fultimo_cierre_total,
    suma_sin_null(Master_mpagado, Visa_mpagado) as mpagado_total,
    suma_sin_null(Master_mpagospesos, Visa_mpagospesos) as mpagospesos_total,
    suma_sin_null(Master_mpagosdolares, Visa_mpagosdolares) as mpagosdolares_total,
    suma_sin_null(Master_fechaalta, Visa_fechaalta) as fechaalta_total,
    suma_sin_null(Master_mconsumototal, Visa_mconsumototal) as mconsumototal_total,
    suma_sin_null(Master_cconsumos, Visa_cconsumos) as cconsumos_total,
    suma_sin_null(Master_cadelantosefectivo, Visa_cadelantosefectivo) as cadelantosefectivo_total,
    suma_sin_null(Master_mpagominimo, Visa_mpagominimo) as mpagominimo_total,
    
    suma_sin_null(ctarjeta_visa, ctarjeta_master) as cant_tarjetas,
    suma_sin_null(ctarjeta_visa_transacciones, ctarjeta_master_transacciones) as cant_transac_tarjetas,
    suma_sin_null(mtarjeta_visa_consumo, mtarjeta_master_consumo) as mtarjetas_consumo,
    suma_sin_null(cprestamos_personales,cprestamos_prendarios) as c_prestamos,
    suma_sin_null(mpayroll, mpayroll2) as m_payroll_total, -- monto acreditacion
    suma_sin_null(cpayroll_trx, cpayroll2_trx) as m_cpayroll_total, -- cantidad de acreditaciones
    suma_sin_null(minversion1_dolares, minversion1_pesos) as minversion1_total,
    suma_sin_null(mplazo_fijo_dolares, mplazo_fijo_pesos) as mplazofijo_total,
    suma_sin_null(suma_sin_null(cseguro_vida, cseguro_auto), suma_sin_null(cseguro_vivienda, cseguro_accidentes_personales)) as cseguros_total,

    div_sin_cero(mautoservicio, ctarjeta_debito_transacciones) as m_transaccion, 
    (ifnull(cliente_antiguedad, 0) / 12.0) as cliente_antiguedad_años,
    div_sin_cero(cliente_antiguedad_años, cliente_edad)  as antiguedad_edad,
    div_sin_cero(mconsumototal_total, mlimitecompra_total) as ratio_uso_credito,
    div_sin_cero(mpagado_total, mconsumototal_total) as ratio_pago_vs_consumo,
    div_sin_cero(mpagominimo_total, mpagado_total) as ratio_pago_min_vs_total,
    div_sin_cero(cant_transac_tarjetas, cant_tarjetas) as transacciones_por_tarjeta,
    div_sin_cero(mconsumospesos_total, mconsumosdolares_total) as ratio_consumo_pesos_dolares,
    div_sin_cero(madelantopesos_total + madelantodolares_total, mlimitecompra_total) as ratio_adelantos_vs_limite,
    div_sin_cero(c_prestamos, m_payroll_total) as ratio_prestamos_vs_acreditaciones,
    div_sin_cero(cseguros_total, mconsumototal_total) as ratio_seguros_vs_consumo,
    div_sin_cero(minversion1_total, msaldototal_total) as ratio_inversiones_vs_saldo
from competencia_01


,Success


In [10]:
%%sql
select 
    *
from competencia_01_suma_division


,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,antiguedad_edad,ratio_uso_credito,ratio_pago_vs_consumo,ratio_pago_min_vs_total,transacciones_por_tarjeta,ratio_consumo_pesos_dolares,ratio_adelantos_vs_limite,ratio_prestamos_vs_acreditaciones,ratio_seguros_vs_consumo,ratio_inversiones_vs_saldo
0,249619949,202101,1,0,0,45,68,-4732.12,-19793.21,295.29,...,0.125926,0.016731,0.000000,NaN,8.0,238.777348,0.0,NaN,0.000056,0.0
1,249619949,202102,1,0,0,45,69,-4565.57,-21930.03,681.57,...,0.127778,0.012747,3.654751,0.235829,10.5,NaN,0.0,NaN,0.000073,0.0
2,249619949,202103,1,0,0,45,70,-3993.18,-23437.50,1881.56,...,0.129630,0.061616,0.000000,NaN,14.0,1942.124926,0.0,NaN,0.000015,0.0
3,249619949,202104,1,0,0,45,71,-4296.05,-26240.03,3213.29,...,0.131481,0.037081,0.000000,NaN,15.0,NaN,0.0,NaN,0.000032,0.0
4,249619949,202105,1,0,0,45,72,-8159.42,-33129.26,837.17,...,0.133333,0.007932,0.000000,NaN,8.0,NaN,0.0,0.0,0.000149,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
981941,585750413,202106,1,0,0,41,168,630.95,17727.46,671.77,...,0.341463,0.032209,0.000000,NaN,11.5,NaN,0.0,0.0,0.000026,0.0
981942,586159235,202101,1,0,0,35,200,-4026.94,-7188.92,866.82,...,0.476190,0.037061,0.000000,NaN,21.5,10924.931937,0.0,NaN,0.000000,0.0
981943,586159235,202102,1,0,0,35,201,-4514.31,-13983.59,406.33,...,0.478571,0.053673,0.000000,NaN,24.5,1680.755284,0.0,NaN,0.000000,0.0
981944,586159235,202103,1,0,0,35,202,-3758.02,-14301.48,561.52,...,0.480952,0.068209,0.000000,NaN,26.5,21335.894444,0.0,NaN,0.000000,0.0


## lag, valor del mes anterior 
## delta diferencia entre meses

In [51]:
# campos = ["ctrx_quarter",
#     'mfinanciacion_limite_total', 
#     'mconsumototal_total',
#       'mconsumospesos_total',
#         'mprestamos_personales',
#         'mconsumosdolares_total',
#           'mpagado_total', 'mpagospesos_total','mcuentas_saldo','mcaja_ahorro',
#     'mpagosdolares_total', 'mpasivos_margen','madelantopesos_total', 'madelantodolares_total', 
#     'ratio_uso_credito', 'ratio_pago_vs_consumo', 'ratio_pago_min_vs_total', 
#     'ratio_adelantos_vs_limite', 'mplazofijo_total',"m_cpayroll_total", 'c_prestamos', 
#     'm_payroll_total', 'ratio_inversiones_vs_saldo', 'minversion1_total', 'mcomisiones_mantenimiento',
#     'mpagominimo_total', 'cseguros_total', 'cant_transac_tarjetas','cproductos','cdescubierto_preacordado','msaldopesos_total',
#     'transacciones_por_tarjeta', 'antiguedad_edad','mrentabilidad_annual_no_null','mrentabilidad_no_null','mtarjetas_consumo','msaldodolares_total'
# ]

In [53]:
nuevos_features = ""
for campo in campos:
  nuevos_features += f"\n    , lag({campo}, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_{campo}"
  nuevos_features += f"\n    , {campo} - lag_1_{campo} as delta_1_{campo}"
  nuevos_features += f"\n    , lag({campo}, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_{campo}"
  nuevos_features += f"\n    , {campo} - lag_2_{campo} as delta_2_{campo}"


consulta_sql = f"""
CREATE OR REPLACE TABLE competencia_01_lag_delta AS
select
    *,
    {nuevos_features.strip()}
FROM competencia_01
"""


In [54]:
print(consulta_sql)


CREATE OR REPLACE TABLE competencia_01_lag_delta AS
select
    numero_de_cliente
    , foto_mes 
    , lag(numero_de_cliente, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_numero_de_cliente
    , numero_de_cliente - lag_1_numero_de_cliente as delta_1_numero_de_cliente
    , lag(numero_de_cliente, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_numero_de_cliente
    , numero_de_cliente - lag_2_numero_de_cliente as delta_2_numero_de_cliente
    , lag(foto_mes, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_foto_mes
    , foto_mes - lag_1_foto_mes as delta_1_foto_mes
    , lag(foto_mes, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_foto_mes
    , foto_mes - lag_2_foto_mes as delta_2_foto_mes
    , lag(active_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_active_quarter
    , active_quarter - lag_1_active_quarter as delta_1_active_quarter
    , lag(active_quarter, 2) over (pa

In [11]:
%%sql
CREATE OR REPLACE TABLE competencia_01_lag_delta AS
select
    numero_de_cliente
    , foto_mes
    , lag(numero_de_cliente, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_numero_de_cliente
    , numero_de_cliente - lag_1_numero_de_cliente as delta_1_numero_de_cliente
    , lag(numero_de_cliente, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_numero_de_cliente
    , numero_de_cliente - lag_2_numero_de_cliente as delta_2_numero_de_cliente
    , lag(foto_mes, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_foto_mes
    , foto_mes - lag_1_foto_mes as delta_1_foto_mes
    , lag(foto_mes, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_foto_mes
    , foto_mes - lag_2_foto_mes as delta_2_foto_mes
    , lag(active_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_active_quarter
    , active_quarter - lag_1_active_quarter as delta_1_active_quarter
    , lag(active_quarter, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_active_quarter
    , active_quarter - lag_2_active_quarter as delta_2_active_quarter
    , lag(cliente_vip, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cliente_vip
    , cliente_vip - lag_1_cliente_vip as delta_1_cliente_vip
    , lag(cliente_vip, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cliente_vip
    , cliente_vip - lag_2_cliente_vip as delta_2_cliente_vip
    , lag(internet, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_internet
    , internet - lag_1_internet as delta_1_internet
    , lag(internet, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_internet
    , internet - lag_2_internet as delta_2_internet
    , lag(cliente_edad, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cliente_edad
    , cliente_edad - lag_1_cliente_edad as delta_1_cliente_edad
    , lag(cliente_edad, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cliente_edad
    , cliente_edad - lag_2_cliente_edad as delta_2_cliente_edad
    , lag(cliente_antiguedad, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cliente_antiguedad
    , cliente_antiguedad - lag_1_cliente_antiguedad as delta_1_cliente_antiguedad
    , lag(cliente_antiguedad, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cliente_antiguedad
    , cliente_antiguedad - lag_2_cliente_antiguedad as delta_2_cliente_antiguedad
    , lag(mrentabilidad, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mrentabilidad
    , mrentabilidad - lag_1_mrentabilidad as delta_1_mrentabilidad
    , lag(mrentabilidad, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mrentabilidad
    , mrentabilidad - lag_2_mrentabilidad as delta_2_mrentabilidad
    , lag(mrentabilidad_annual, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mrentabilidad_annual
    , mrentabilidad_annual - lag_1_mrentabilidad_annual as delta_1_mrentabilidad_annual
    , lag(mrentabilidad_annual, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mrentabilidad_annual
    , mrentabilidad_annual - lag_2_mrentabilidad_annual as delta_2_mrentabilidad_annual
    , lag(mcomisiones, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcomisiones
    , mcomisiones - lag_1_mcomisiones as delta_1_mcomisiones
    , lag(mcomisiones, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mcomisiones
    , mcomisiones - lag_2_mcomisiones as delta_2_mcomisiones
    , lag(mactivos_margen, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mactivos_margen
    , mactivos_margen - lag_1_mactivos_margen as delta_1_mactivos_margen
    , lag(mactivos_margen, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mactivos_margen
    , mactivos_margen - lag_2_mactivos_margen as delta_2_mactivos_margen
    , lag(mpasivos_margen, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mpasivos_margen
    , mpasivos_margen - lag_1_mpasivos_margen as delta_1_mpasivos_margen
    , lag(mpasivos_margen, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mpasivos_margen
    , mpasivos_margen - lag_2_mpasivos_margen as delta_2_mpasivos_margen
    , lag(cproductos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cproductos
    , cproductos - lag_1_cproductos as delta_1_cproductos
    , lag(cproductos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cproductos
    , cproductos - lag_2_cproductos as delta_2_cproductos
    , lag(tcuentas, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_tcuentas
    , tcuentas - lag_1_tcuentas as delta_1_tcuentas
    , lag(tcuentas, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_tcuentas
    , tcuentas - lag_2_tcuentas as delta_2_tcuentas
    , lag(ccuenta_corriente, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccuenta_corriente
    , ccuenta_corriente - lag_1_ccuenta_corriente as delta_1_ccuenta_corriente
    , lag(ccuenta_corriente, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ccuenta_corriente
    , ccuenta_corriente - lag_2_ccuenta_corriente as delta_2_ccuenta_corriente
    , lag(mcuenta_corriente_adicional, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcuenta_corriente_adicional
    , mcuenta_corriente_adicional - lag_1_mcuenta_corriente_adicional as delta_1_mcuenta_corriente_adicional
    , lag(mcuenta_corriente_adicional, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mcuenta_corriente_adicional
    , mcuenta_corriente_adicional - lag_2_mcuenta_corriente_adicional as delta_2_mcuenta_corriente_adicional
    , lag(mcuenta_corriente, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcuenta_corriente
    , mcuenta_corriente - lag_1_mcuenta_corriente as delta_1_mcuenta_corriente
    , lag(mcuenta_corriente, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mcuenta_corriente
    , mcuenta_corriente - lag_2_mcuenta_corriente as delta_2_mcuenta_corriente
    , lag(ccaja_ahorro, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccaja_ahorro
    , ccaja_ahorro - lag_1_ccaja_ahorro as delta_1_ccaja_ahorro
    , lag(ccaja_ahorro, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ccaja_ahorro
    , ccaja_ahorro - lag_2_ccaja_ahorro as delta_2_ccaja_ahorro
    , lag(mcaja_ahorro, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcaja_ahorro
    , mcaja_ahorro - lag_1_mcaja_ahorro as delta_1_mcaja_ahorro
    , lag(mcaja_ahorro, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mcaja_ahorro
    , mcaja_ahorro - lag_2_mcaja_ahorro as delta_2_mcaja_ahorro
    , lag(mcaja_ahorro_adicional, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcaja_ahorro_adicional
    , mcaja_ahorro_adicional - lag_1_mcaja_ahorro_adicional as delta_1_mcaja_ahorro_adicional
    , lag(mcaja_ahorro_adicional, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mcaja_ahorro_adicional
    , mcaja_ahorro_adicional - lag_2_mcaja_ahorro_adicional as delta_2_mcaja_ahorro_adicional
    , lag(mcaja_ahorro_dolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcaja_ahorro_dolares
    , mcaja_ahorro_dolares - lag_1_mcaja_ahorro_dolares as delta_1_mcaja_ahorro_dolares
    , lag(mcaja_ahorro_dolares, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mcaja_ahorro_dolares
    , mcaja_ahorro_dolares - lag_2_mcaja_ahorro_dolares as delta_2_mcaja_ahorro_dolares
    , lag(cdescubierto_preacordado, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cdescubierto_preacordado
    , cdescubierto_preacordado - lag_1_cdescubierto_preacordado as delta_1_cdescubierto_preacordado
    , lag(cdescubierto_preacordado, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cdescubierto_preacordado
    , cdescubierto_preacordado - lag_2_cdescubierto_preacordado as delta_2_cdescubierto_preacordado
    , lag(mcuentas_saldo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcuentas_saldo
    , mcuentas_saldo - lag_1_mcuentas_saldo as delta_1_mcuentas_saldo
    , lag(mcuentas_saldo, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mcuentas_saldo
    , mcuentas_saldo - lag_2_mcuentas_saldo as delta_2_mcuentas_saldo
    , lag(ctarjeta_debito, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctarjeta_debito
    , ctarjeta_debito - lag_1_ctarjeta_debito as delta_1_ctarjeta_debito
    , lag(ctarjeta_debito, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctarjeta_debito
    , ctarjeta_debito - lag_2_ctarjeta_debito as delta_2_ctarjeta_debito
    , lag(ctarjeta_debito_transacciones, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctarjeta_debito_transacciones
    , ctarjeta_debito_transacciones - lag_1_ctarjeta_debito_transacciones as delta_1_ctarjeta_debito_transacciones
    , lag(ctarjeta_debito_transacciones, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctarjeta_debito_transacciones
    , ctarjeta_debito_transacciones - lag_2_ctarjeta_debito_transacciones as delta_2_ctarjeta_debito_transacciones
    , lag(mautoservicio, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mautoservicio
    , mautoservicio - lag_1_mautoservicio as delta_1_mautoservicio
    , lag(mautoservicio, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mautoservicio
    , mautoservicio - lag_2_mautoservicio as delta_2_mautoservicio
    , lag(ctarjeta_visa, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctarjeta_visa
    , ctarjeta_visa - lag_1_ctarjeta_visa as delta_1_ctarjeta_visa
    , lag(ctarjeta_visa, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctarjeta_visa
    , ctarjeta_visa - lag_2_ctarjeta_visa as delta_2_ctarjeta_visa
    , lag(ctarjeta_visa_transacciones, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctarjeta_visa_transacciones
    , ctarjeta_visa_transacciones - lag_1_ctarjeta_visa_transacciones as delta_1_ctarjeta_visa_transacciones
    , lag(ctarjeta_visa_transacciones, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctarjeta_visa_transacciones
    , ctarjeta_visa_transacciones - lag_2_ctarjeta_visa_transacciones as delta_2_ctarjeta_visa_transacciones
    , lag(mtarjeta_visa_consumo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mtarjeta_visa_consumo
    , mtarjeta_visa_consumo - lag_1_mtarjeta_visa_consumo as delta_1_mtarjeta_visa_consumo
    , lag(mtarjeta_visa_consumo, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mtarjeta_visa_consumo
    , mtarjeta_visa_consumo - lag_2_mtarjeta_visa_consumo as delta_2_mtarjeta_visa_consumo
    , lag(ctarjeta_master, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctarjeta_master
    , ctarjeta_master - lag_1_ctarjeta_master as delta_1_ctarjeta_master
    , lag(ctarjeta_master, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctarjeta_master
    , ctarjeta_master - lag_2_ctarjeta_master as delta_2_ctarjeta_master
    , lag(ctarjeta_master_transacciones, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctarjeta_master_transacciones
    , ctarjeta_master_transacciones - lag_1_ctarjeta_master_transacciones as delta_1_ctarjeta_master_transacciones
    , lag(ctarjeta_master_transacciones, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctarjeta_master_transacciones
    , ctarjeta_master_transacciones - lag_2_ctarjeta_master_transacciones as delta_2_ctarjeta_master_transacciones
    , lag(mtarjeta_master_consumo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mtarjeta_master_consumo
    , mtarjeta_master_consumo - lag_1_mtarjeta_master_consumo as delta_1_mtarjeta_master_consumo
    , lag(mtarjeta_master_consumo, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mtarjeta_master_consumo
    , mtarjeta_master_consumo - lag_2_mtarjeta_master_consumo as delta_2_mtarjeta_master_consumo
    , lag(cprestamos_personales, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cprestamos_personales
    , cprestamos_personales - lag_1_cprestamos_personales as delta_1_cprestamos_personales
    , lag(cprestamos_personales, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cprestamos_personales
    , cprestamos_personales - lag_2_cprestamos_personales as delta_2_cprestamos_personales
    , lag(mprestamos_personales, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mprestamos_personales
    , mprestamos_personales - lag_1_mprestamos_personales as delta_1_mprestamos_personales
    , lag(mprestamos_personales, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mprestamos_personales
    , mprestamos_personales - lag_2_mprestamos_personales as delta_2_mprestamos_personales
    , lag(cprestamos_prendarios, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cprestamos_prendarios
    , cprestamos_prendarios - lag_1_cprestamos_prendarios as delta_1_cprestamos_prendarios
    , lag(cprestamos_prendarios, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cprestamos_prendarios
    , cprestamos_prendarios - lag_2_cprestamos_prendarios as delta_2_cprestamos_prendarios
    , lag(mprestamos_prendarios, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mprestamos_prendarios
    , mprestamos_prendarios - lag_1_mprestamos_prendarios as delta_1_mprestamos_prendarios
    , lag(mprestamos_prendarios, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mprestamos_prendarios
    , mprestamos_prendarios - lag_2_mprestamos_prendarios as delta_2_mprestamos_prendarios
    , lag(cprestamos_hipotecarios, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cprestamos_hipotecarios
    , cprestamos_hipotecarios - lag_1_cprestamos_hipotecarios as delta_1_cprestamos_hipotecarios
    , lag(cprestamos_hipotecarios, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cprestamos_hipotecarios
    , cprestamos_hipotecarios - lag_2_cprestamos_hipotecarios as delta_2_cprestamos_hipotecarios
    , lag(mprestamos_hipotecarios, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mprestamos_hipotecarios
    , mprestamos_hipotecarios - lag_1_mprestamos_hipotecarios as delta_1_mprestamos_hipotecarios
    , lag(mprestamos_hipotecarios, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mprestamos_hipotecarios
    , mprestamos_hipotecarios - lag_2_mprestamos_hipotecarios as delta_2_mprestamos_hipotecarios
    , lag(cplazo_fijo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cplazo_fijo
    , cplazo_fijo - lag_1_cplazo_fijo as delta_1_cplazo_fijo
    , lag(cplazo_fijo, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cplazo_fijo
    , cplazo_fijo - lag_2_cplazo_fijo as delta_2_cplazo_fijo
    , lag(mplazo_fijo_dolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mplazo_fijo_dolares
    , mplazo_fijo_dolares - lag_1_mplazo_fijo_dolares as delta_1_mplazo_fijo_dolares
    , lag(mplazo_fijo_dolares, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mplazo_fijo_dolares
    , mplazo_fijo_dolares - lag_2_mplazo_fijo_dolares as delta_2_mplazo_fijo_dolares
    , lag(mplazo_fijo_pesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mplazo_fijo_pesos
    , mplazo_fijo_pesos - lag_1_mplazo_fijo_pesos as delta_1_mplazo_fijo_pesos
    , lag(mplazo_fijo_pesos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mplazo_fijo_pesos
    , mplazo_fijo_pesos - lag_2_mplazo_fijo_pesos as delta_2_mplazo_fijo_pesos
    , lag(cinversion1, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cinversion1
    , cinversion1 - lag_1_cinversion1 as delta_1_cinversion1
    , lag(cinversion1, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cinversion1
    , cinversion1 - lag_2_cinversion1 as delta_2_cinversion1
    , lag(minversion1_pesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_minversion1_pesos
    , minversion1_pesos - lag_1_minversion1_pesos as delta_1_minversion1_pesos
    , lag(minversion1_pesos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_minversion1_pesos
    , minversion1_pesos - lag_2_minversion1_pesos as delta_2_minversion1_pesos
    , lag(minversion1_dolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_minversion1_dolares
    , minversion1_dolares - lag_1_minversion1_dolares as delta_1_minversion1_dolares
    , lag(minversion1_dolares, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_minversion1_dolares
    , minversion1_dolares - lag_2_minversion1_dolares as delta_2_minversion1_dolares
    , lag(cinversion2, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cinversion2
    , cinversion2 - lag_1_cinversion2 as delta_1_cinversion2
    , lag(cinversion2, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cinversion2
    , cinversion2 - lag_2_cinversion2 as delta_2_cinversion2
    , lag(minversion2, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_minversion2
    , minversion2 - lag_1_minversion2 as delta_1_minversion2
    , lag(minversion2, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_minversion2
    , minversion2 - lag_2_minversion2 as delta_2_minversion2
    , lag(cseguro_vida, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cseguro_vida
    , cseguro_vida - lag_1_cseguro_vida as delta_1_cseguro_vida
    , lag(cseguro_vida, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cseguro_vida
    , cseguro_vida - lag_2_cseguro_vida as delta_2_cseguro_vida
    , lag(cseguro_auto, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cseguro_auto
    , cseguro_auto - lag_1_cseguro_auto as delta_1_cseguro_auto
    , lag(cseguro_auto, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cseguro_auto
    , cseguro_auto - lag_2_cseguro_auto as delta_2_cseguro_auto
    , lag(cseguro_vivienda, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cseguro_vivienda
    , cseguro_vivienda - lag_1_cseguro_vivienda as delta_1_cseguro_vivienda
    , lag(cseguro_vivienda, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cseguro_vivienda
    , cseguro_vivienda - lag_2_cseguro_vivienda as delta_2_cseguro_vivienda
    , lag(cseguro_accidentes_personales, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cseguro_accidentes_personales
    , cseguro_accidentes_personales - lag_1_cseguro_accidentes_personales as delta_1_cseguro_accidentes_personales
    , lag(cseguro_accidentes_personales, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cseguro_accidentes_personales
    , cseguro_accidentes_personales - lag_2_cseguro_accidentes_personales as delta_2_cseguro_accidentes_personales
    , lag(ccaja_seguridad, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccaja_seguridad
    , ccaja_seguridad - lag_1_ccaja_seguridad as delta_1_ccaja_seguridad
    , lag(ccaja_seguridad, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ccaja_seguridad
    , ccaja_seguridad - lag_2_ccaja_seguridad as delta_2_ccaja_seguridad
    , lag(cpayroll_trx, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cpayroll_trx
    , cpayroll_trx - lag_1_cpayroll_trx as delta_1_cpayroll_trx
    , lag(cpayroll_trx, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cpayroll_trx
    , cpayroll_trx - lag_2_cpayroll_trx as delta_2_cpayroll_trx
    , lag(mpayroll, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mpayroll
    , mpayroll - lag_1_mpayroll as delta_1_mpayroll
    , lag(mpayroll, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mpayroll
    , mpayroll - lag_2_mpayroll as delta_2_mpayroll
    , lag(mpayroll2, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mpayroll2
    , mpayroll2 - lag_1_mpayroll2 as delta_1_mpayroll2
    , lag(mpayroll2, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mpayroll2
    , mpayroll2 - lag_2_mpayroll2 as delta_2_mpayroll2
    , lag(cpayroll2_trx, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cpayroll2_trx
    , cpayroll2_trx - lag_1_cpayroll2_trx as delta_1_cpayroll2_trx
    , lag(cpayroll2_trx, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cpayroll2_trx
    , cpayroll2_trx - lag_2_cpayroll2_trx as delta_2_cpayroll2_trx
    , lag(ccuenta_debitos_automaticos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccuenta_debitos_automaticos
    , ccuenta_debitos_automaticos - lag_1_ccuenta_debitos_automaticos as delta_1_ccuenta_debitos_automaticos
    , lag(ccuenta_debitos_automaticos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ccuenta_debitos_automaticos
    , ccuenta_debitos_automaticos - lag_2_ccuenta_debitos_automaticos as delta_2_ccuenta_debitos_automaticos
    , lag(mcuenta_debitos_automaticos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcuenta_debitos_automaticos
    , mcuenta_debitos_automaticos - lag_1_mcuenta_debitos_automaticos as delta_1_mcuenta_debitos_automaticos
    , lag(mcuenta_debitos_automaticos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mcuenta_debitos_automaticos
    , mcuenta_debitos_automaticos - lag_2_mcuenta_debitos_automaticos as delta_2_mcuenta_debitos_automaticos
    , lag(ctarjeta_visa_debitos_automaticos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctarjeta_visa_debitos_automaticos
    , ctarjeta_visa_debitos_automaticos - lag_1_ctarjeta_visa_debitos_automaticos as delta_1_ctarjeta_visa_debitos_automaticos
    , lag(ctarjeta_visa_debitos_automaticos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctarjeta_visa_debitos_automaticos
    , ctarjeta_visa_debitos_automaticos - lag_2_ctarjeta_visa_debitos_automaticos as delta_2_ctarjeta_visa_debitos_automaticos
    , lag(mttarjeta_visa_debitos_automaticos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mttarjeta_visa_debitos_automaticos
    , mttarjeta_visa_debitos_automaticos - lag_1_mttarjeta_visa_debitos_automaticos as delta_1_mttarjeta_visa_debitos_automaticos
    , lag(mttarjeta_visa_debitos_automaticos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mttarjeta_visa_debitos_automaticos
    , mttarjeta_visa_debitos_automaticos - lag_2_mttarjeta_visa_debitos_automaticos as delta_2_mttarjeta_visa_debitos_automaticos
    , lag(ctarjeta_master_debitos_automaticos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctarjeta_master_debitos_automaticos
    , ctarjeta_master_debitos_automaticos - lag_1_ctarjeta_master_debitos_automaticos as delta_1_ctarjeta_master_debitos_automaticos
    , lag(ctarjeta_master_debitos_automaticos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctarjeta_master_debitos_automaticos
    , ctarjeta_master_debitos_automaticos - lag_2_ctarjeta_master_debitos_automaticos as delta_2_ctarjeta_master_debitos_automaticos
    , lag(mttarjeta_master_debitos_automaticos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mttarjeta_master_debitos_automaticos
    , mttarjeta_master_debitos_automaticos - lag_1_mttarjeta_master_debitos_automaticos as delta_1_mttarjeta_master_debitos_automaticos
    , lag(mttarjeta_master_debitos_automaticos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mttarjeta_master_debitos_automaticos
    , mttarjeta_master_debitos_automaticos - lag_2_mttarjeta_master_debitos_automaticos as delta_2_mttarjeta_master_debitos_automaticos
    , lag(cpagodeservicios, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cpagodeservicios
    , cpagodeservicios - lag_1_cpagodeservicios as delta_1_cpagodeservicios
    , lag(cpagodeservicios, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cpagodeservicios
    , cpagodeservicios - lag_2_cpagodeservicios as delta_2_cpagodeservicios
    , lag(mpagodeservicios, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mpagodeservicios
    , mpagodeservicios - lag_1_mpagodeservicios as delta_1_mpagodeservicios
    , lag(mpagodeservicios, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mpagodeservicios
    , mpagodeservicios - lag_2_mpagodeservicios as delta_2_mpagodeservicios
    , lag(cpagomiscuentas, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cpagomiscuentas
    , cpagomiscuentas - lag_1_cpagomiscuentas as delta_1_cpagomiscuentas
    , lag(cpagomiscuentas, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cpagomiscuentas
    , cpagomiscuentas - lag_2_cpagomiscuentas as delta_2_cpagomiscuentas
    , lag(mpagomiscuentas, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mpagomiscuentas
    , mpagomiscuentas - lag_1_mpagomiscuentas as delta_1_mpagomiscuentas
    , lag(mpagomiscuentas, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mpagomiscuentas
    , mpagomiscuentas - lag_2_mpagomiscuentas as delta_2_mpagomiscuentas
    , lag(ccajeros_propios_descuentos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccajeros_propios_descuentos
    , ccajeros_propios_descuentos - lag_1_ccajeros_propios_descuentos as delta_1_ccajeros_propios_descuentos
    , lag(ccajeros_propios_descuentos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ccajeros_propios_descuentos
    , ccajeros_propios_descuentos - lag_2_ccajeros_propios_descuentos as delta_2_ccajeros_propios_descuentos
    , lag(mcajeros_propios_descuentos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcajeros_propios_descuentos
    , mcajeros_propios_descuentos - lag_1_mcajeros_propios_descuentos as delta_1_mcajeros_propios_descuentos
    , lag(mcajeros_propios_descuentos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mcajeros_propios_descuentos
    , mcajeros_propios_descuentos - lag_2_mcajeros_propios_descuentos as delta_2_mcajeros_propios_descuentos
    , lag(ctarjeta_visa_descuentos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctarjeta_visa_descuentos
    , ctarjeta_visa_descuentos - lag_1_ctarjeta_visa_descuentos as delta_1_ctarjeta_visa_descuentos
    , lag(ctarjeta_visa_descuentos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctarjeta_visa_descuentos
    , ctarjeta_visa_descuentos - lag_2_ctarjeta_visa_descuentos as delta_2_ctarjeta_visa_descuentos
    , lag(mtarjeta_visa_descuentos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mtarjeta_visa_descuentos
    , mtarjeta_visa_descuentos - lag_1_mtarjeta_visa_descuentos as delta_1_mtarjeta_visa_descuentos
    , lag(mtarjeta_visa_descuentos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mtarjeta_visa_descuentos
    , mtarjeta_visa_descuentos - lag_2_mtarjeta_visa_descuentos as delta_2_mtarjeta_visa_descuentos
    , lag(ctarjeta_master_descuentos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctarjeta_master_descuentos
    , ctarjeta_master_descuentos - lag_1_ctarjeta_master_descuentos as delta_1_ctarjeta_master_descuentos
    , lag(ctarjeta_master_descuentos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctarjeta_master_descuentos
    , ctarjeta_master_descuentos - lag_2_ctarjeta_master_descuentos as delta_2_ctarjeta_master_descuentos
    , lag(mtarjeta_master_descuentos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mtarjeta_master_descuentos
    , mtarjeta_master_descuentos - lag_1_mtarjeta_master_descuentos as delta_1_mtarjeta_master_descuentos
    , lag(mtarjeta_master_descuentos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mtarjeta_master_descuentos
    , mtarjeta_master_descuentos - lag_2_mtarjeta_master_descuentos as delta_2_mtarjeta_master_descuentos
    , lag(ccomisiones_mantenimiento, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccomisiones_mantenimiento
    , ccomisiones_mantenimiento - lag_1_ccomisiones_mantenimiento as delta_1_ccomisiones_mantenimiento
    , lag(ccomisiones_mantenimiento, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ccomisiones_mantenimiento
    , ccomisiones_mantenimiento - lag_2_ccomisiones_mantenimiento as delta_2_ccomisiones_mantenimiento
    , lag(mcomisiones_mantenimiento, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcomisiones_mantenimiento
    , mcomisiones_mantenimiento - lag_1_mcomisiones_mantenimiento as delta_1_mcomisiones_mantenimiento
    , lag(mcomisiones_mantenimiento, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mcomisiones_mantenimiento
    , mcomisiones_mantenimiento - lag_2_mcomisiones_mantenimiento as delta_2_mcomisiones_mantenimiento
    , lag(ccomisiones_otras, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccomisiones_otras
    , ccomisiones_otras - lag_1_ccomisiones_otras as delta_1_ccomisiones_otras
    , lag(ccomisiones_otras, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ccomisiones_otras
    , ccomisiones_otras - lag_2_ccomisiones_otras as delta_2_ccomisiones_otras
    , lag(mcomisiones_otras, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcomisiones_otras
    , mcomisiones_otras - lag_1_mcomisiones_otras as delta_1_mcomisiones_otras
    , lag(mcomisiones_otras, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mcomisiones_otras
    , mcomisiones_otras - lag_2_mcomisiones_otras as delta_2_mcomisiones_otras
    , lag(cforex, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cforex
    , cforex - lag_1_cforex as delta_1_cforex
    , lag(cforex, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cforex
    , cforex - lag_2_cforex as delta_2_cforex
    , lag(cforex_buy, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cforex_buy
    , cforex_buy - lag_1_cforex_buy as delta_1_cforex_buy
    , lag(cforex_buy, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cforex_buy
    , cforex_buy - lag_2_cforex_buy as delta_2_cforex_buy
    , lag(mforex_buy, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mforex_buy
    , mforex_buy - lag_1_mforex_buy as delta_1_mforex_buy
    , lag(mforex_buy, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mforex_buy
    , mforex_buy - lag_2_mforex_buy as delta_2_mforex_buy
    , lag(cforex_sell, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cforex_sell
    , cforex_sell - lag_1_cforex_sell as delta_1_cforex_sell
    , lag(cforex_sell, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cforex_sell
    , cforex_sell - lag_2_cforex_sell as delta_2_cforex_sell
    , lag(mforex_sell, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mforex_sell
    , mforex_sell - lag_1_mforex_sell as delta_1_mforex_sell
    , lag(mforex_sell, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mforex_sell
    , mforex_sell - lag_2_mforex_sell as delta_2_mforex_sell
    , lag(ctransferencias_recibidas, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctransferencias_recibidas
    , ctransferencias_recibidas - lag_1_ctransferencias_recibidas as delta_1_ctransferencias_recibidas
    , lag(ctransferencias_recibidas, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctransferencias_recibidas
    , ctransferencias_recibidas - lag_2_ctransferencias_recibidas as delta_2_ctransferencias_recibidas
    , lag(mtransferencias_recibidas, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mtransferencias_recibidas
    , mtransferencias_recibidas - lag_1_mtransferencias_recibidas as delta_1_mtransferencias_recibidas
    , lag(mtransferencias_recibidas, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mtransferencias_recibidas
    , mtransferencias_recibidas - lag_2_mtransferencias_recibidas as delta_2_mtransferencias_recibidas
    , lag(ctransferencias_emitidas, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctransferencias_emitidas
    , ctransferencias_emitidas - lag_1_ctransferencias_emitidas as delta_1_ctransferencias_emitidas
    , lag(ctransferencias_emitidas, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctransferencias_emitidas
    , ctransferencias_emitidas - lag_2_ctransferencias_emitidas as delta_2_ctransferencias_emitidas
    , lag(mtransferencias_emitidas, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mtransferencias_emitidas
    , mtransferencias_emitidas - lag_1_mtransferencias_emitidas as delta_1_mtransferencias_emitidas
    , lag(mtransferencias_emitidas, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mtransferencias_emitidas
    , mtransferencias_emitidas - lag_2_mtransferencias_emitidas as delta_2_mtransferencias_emitidas
    , lag(cextraccion_autoservicio, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cextraccion_autoservicio
    , cextraccion_autoservicio - lag_1_cextraccion_autoservicio as delta_1_cextraccion_autoservicio
    , lag(cextraccion_autoservicio, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cextraccion_autoservicio
    , cextraccion_autoservicio - lag_2_cextraccion_autoservicio as delta_2_cextraccion_autoservicio
    , lag(mextraccion_autoservicio, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mextraccion_autoservicio
    , mextraccion_autoservicio - lag_1_mextraccion_autoservicio as delta_1_mextraccion_autoservicio
    , lag(mextraccion_autoservicio, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mextraccion_autoservicio
    , mextraccion_autoservicio - lag_2_mextraccion_autoservicio as delta_2_mextraccion_autoservicio
    , lag(ccheques_depositados, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccheques_depositados
    , ccheques_depositados - lag_1_ccheques_depositados as delta_1_ccheques_depositados
    , lag(ccheques_depositados, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ccheques_depositados
    , ccheques_depositados - lag_2_ccheques_depositados as delta_2_ccheques_depositados
    , lag(mcheques_depositados, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcheques_depositados
    , mcheques_depositados - lag_1_mcheques_depositados as delta_1_mcheques_depositados
    , lag(mcheques_depositados, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mcheques_depositados
    , mcheques_depositados - lag_2_mcheques_depositados as delta_2_mcheques_depositados
    , lag(ccheques_emitidos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccheques_emitidos
    , ccheques_emitidos - lag_1_ccheques_emitidos as delta_1_ccheques_emitidos
    , lag(ccheques_emitidos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ccheques_emitidos
    , ccheques_emitidos - lag_2_ccheques_emitidos as delta_2_ccheques_emitidos
    , lag(mcheques_emitidos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcheques_emitidos
    , mcheques_emitidos - lag_1_mcheques_emitidos as delta_1_mcheques_emitidos
    , lag(mcheques_emitidos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mcheques_emitidos
    , mcheques_emitidos - lag_2_mcheques_emitidos as delta_2_mcheques_emitidos
    , lag(ccheques_depositados_rechazados, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccheques_depositados_rechazados
    , ccheques_depositados_rechazados - lag_1_ccheques_depositados_rechazados as delta_1_ccheques_depositados_rechazados
    , lag(ccheques_depositados_rechazados, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ccheques_depositados_rechazados
    , ccheques_depositados_rechazados - lag_2_ccheques_depositados_rechazados as delta_2_ccheques_depositados_rechazados
    , lag(mcheques_depositados_rechazados, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcheques_depositados_rechazados
    , mcheques_depositados_rechazados - lag_1_mcheques_depositados_rechazados as delta_1_mcheques_depositados_rechazados
    , lag(mcheques_depositados_rechazados, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mcheques_depositados_rechazados
    , mcheques_depositados_rechazados - lag_2_mcheques_depositados_rechazados as delta_2_mcheques_depositados_rechazados
    , lag(ccheques_emitidos_rechazados, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccheques_emitidos_rechazados
    , ccheques_emitidos_rechazados - lag_1_ccheques_emitidos_rechazados as delta_1_ccheques_emitidos_rechazados
    , lag(ccheques_emitidos_rechazados, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ccheques_emitidos_rechazados
    , ccheques_emitidos_rechazados - lag_2_ccheques_emitidos_rechazados as delta_2_ccheques_emitidos_rechazados
    , lag(mcheques_emitidos_rechazados, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcheques_emitidos_rechazados
    , mcheques_emitidos_rechazados - lag_1_mcheques_emitidos_rechazados as delta_1_mcheques_emitidos_rechazados
    , lag(mcheques_emitidos_rechazados, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mcheques_emitidos_rechazados
    , mcheques_emitidos_rechazados - lag_2_mcheques_emitidos_rechazados as delta_2_mcheques_emitidos_rechazados
    , lag(tcallcenter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_tcallcenter
    , tcallcenter - lag_1_tcallcenter as delta_1_tcallcenter
    , lag(tcallcenter, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_tcallcenter
    , tcallcenter - lag_2_tcallcenter as delta_2_tcallcenter
    , lag(ccallcenter_transacciones, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccallcenter_transacciones
    , ccallcenter_transacciones - lag_1_ccallcenter_transacciones as delta_1_ccallcenter_transacciones
    , lag(ccallcenter_transacciones, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ccallcenter_transacciones
    , ccallcenter_transacciones - lag_2_ccallcenter_transacciones as delta_2_ccallcenter_transacciones
    , lag(thomebanking, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_thomebanking
    , thomebanking - lag_1_thomebanking as delta_1_thomebanking
    , lag(thomebanking, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_thomebanking
    , thomebanking - lag_2_thomebanking as delta_2_thomebanking
    , lag(chomebanking_transacciones, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_chomebanking_transacciones
    , chomebanking_transacciones - lag_1_chomebanking_transacciones as delta_1_chomebanking_transacciones
    , lag(chomebanking_transacciones, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_chomebanking_transacciones
    , chomebanking_transacciones - lag_2_chomebanking_transacciones as delta_2_chomebanking_transacciones
    , lag(ccajas_transacciones, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccajas_transacciones
    , ccajas_transacciones - lag_1_ccajas_transacciones as delta_1_ccajas_transacciones
    , lag(ccajas_transacciones, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ccajas_transacciones
    , ccajas_transacciones - lag_2_ccajas_transacciones as delta_2_ccajas_transacciones
    , lag(ccajas_consultas, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccajas_consultas
    , ccajas_consultas - lag_1_ccajas_consultas as delta_1_ccajas_consultas
    , lag(ccajas_consultas, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ccajas_consultas
    , ccajas_consultas - lag_2_ccajas_consultas as delta_2_ccajas_consultas
    , lag(ccajas_depositos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccajas_depositos
    , ccajas_depositos - lag_1_ccajas_depositos as delta_1_ccajas_depositos
    , lag(ccajas_depositos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ccajas_depositos
    , ccajas_depositos - lag_2_ccajas_depositos as delta_2_ccajas_depositos
    , lag(ccajas_extracciones, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccajas_extracciones
    , ccajas_extracciones - lag_1_ccajas_extracciones as delta_1_ccajas_extracciones
    , lag(ccajas_extracciones, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ccajas_extracciones
    , ccajas_extracciones - lag_2_ccajas_extracciones as delta_2_ccajas_extracciones
    , lag(ccajas_otras, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccajas_otras
    , ccajas_otras - lag_1_ccajas_otras as delta_1_ccajas_otras
    , lag(ccajas_otras, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ccajas_otras
    , ccajas_otras - lag_2_ccajas_otras as delta_2_ccajas_otras
    , lag(catm_trx, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_catm_trx
    , catm_trx - lag_1_catm_trx as delta_1_catm_trx
    , lag(catm_trx, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_catm_trx
    , catm_trx - lag_2_catm_trx as delta_2_catm_trx
    , lag(matm, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_matm
    , matm - lag_1_matm as delta_1_matm
    , lag(matm, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_matm
    , matm - lag_2_matm as delta_2_matm
    , lag(catm_trx_other, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_catm_trx_other
    , catm_trx_other - lag_1_catm_trx_other as delta_1_catm_trx_other
    , lag(catm_trx_other, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_catm_trx_other
    , catm_trx_other - lag_2_catm_trx_other as delta_2_catm_trx_other
    , lag(matm_other, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_matm_other
    , matm_other - lag_1_matm_other as delta_1_matm_other
    , lag(matm_other, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_matm_other
    , matm_other - lag_2_matm_other as delta_2_matm_other
    , lag(ctrx_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctrx_quarter
    , ctrx_quarter - lag_1_ctrx_quarter as delta_1_ctrx_quarter
    , lag(ctrx_quarter, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctrx_quarter
    , ctrx_quarter - lag_2_ctrx_quarter as delta_2_ctrx_quarter
    , lag(tmobile_app, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_tmobile_app
    , tmobile_app - lag_1_tmobile_app as delta_1_tmobile_app
    , lag(tmobile_app, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_tmobile_app
    , tmobile_app - lag_2_tmobile_app as delta_2_tmobile_app
    , lag(cmobile_app_trx, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cmobile_app_trx
    , cmobile_app_trx - lag_1_cmobile_app_trx as delta_1_cmobile_app_trx
    , lag(cmobile_app_trx, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cmobile_app_trx
    , cmobile_app_trx - lag_2_cmobile_app_trx as delta_2_cmobile_app_trx
    , lag(Master_delinquency, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_delinquency
    , Master_delinquency - lag_1_Master_delinquency as delta_1_Master_delinquency
    , lag(Master_delinquency, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Master_delinquency
    , Master_delinquency - lag_2_Master_delinquency as delta_2_Master_delinquency
    , lag(Master_status, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_status
    , Master_status - lag_1_Master_status as delta_1_Master_status
    , lag(Master_status, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Master_status
    , Master_status - lag_2_Master_status as delta_2_Master_status
    , lag(Master_mfinanciacion_limite, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_mfinanciacion_limite
    , Master_mfinanciacion_limite - lag_1_Master_mfinanciacion_limite as delta_1_Master_mfinanciacion_limite
    , lag(Master_mfinanciacion_limite, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Master_mfinanciacion_limite
    , Master_mfinanciacion_limite - lag_2_Master_mfinanciacion_limite as delta_2_Master_mfinanciacion_limite
    , lag(Master_Fvencimiento, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_Fvencimiento
    , Master_Fvencimiento - lag_1_Master_Fvencimiento as delta_1_Master_Fvencimiento
    , lag(Master_Fvencimiento, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Master_Fvencimiento
    , Master_Fvencimiento - lag_2_Master_Fvencimiento as delta_2_Master_Fvencimiento
    , lag(Master_Finiciomora, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_Finiciomora
    , Master_Finiciomora - lag_1_Master_Finiciomora as delta_1_Master_Finiciomora
    , lag(Master_Finiciomora, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Master_Finiciomora
    , Master_Finiciomora - lag_2_Master_Finiciomora as delta_2_Master_Finiciomora
    , lag(Master_msaldototal, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_msaldototal
    , Master_msaldototal - lag_1_Master_msaldototal as delta_1_Master_msaldototal
    , lag(Master_msaldototal, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Master_msaldototal
    , Master_msaldototal - lag_2_Master_msaldototal as delta_2_Master_msaldototal
    , lag(Master_msaldopesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_msaldopesos
    , Master_msaldopesos - lag_1_Master_msaldopesos as delta_1_Master_msaldopesos
    , lag(Master_msaldopesos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Master_msaldopesos
    , Master_msaldopesos - lag_2_Master_msaldopesos as delta_2_Master_msaldopesos
    , lag(Master_msaldodolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_msaldodolares
    , Master_msaldodolares - lag_1_Master_msaldodolares as delta_1_Master_msaldodolares
    , lag(Master_msaldodolares, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Master_msaldodolares
    , Master_msaldodolares - lag_2_Master_msaldodolares as delta_2_Master_msaldodolares
    , lag(Master_mconsumospesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_mconsumospesos
    , Master_mconsumospesos - lag_1_Master_mconsumospesos as delta_1_Master_mconsumospesos
    , lag(Master_mconsumospesos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Master_mconsumospesos
    , Master_mconsumospesos - lag_2_Master_mconsumospesos as delta_2_Master_mconsumospesos
    , lag(Master_mconsumosdolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_mconsumosdolares
    , Master_mconsumosdolares - lag_1_Master_mconsumosdolares as delta_1_Master_mconsumosdolares
    , lag(Master_mconsumosdolares, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Master_mconsumosdolares
    , Master_mconsumosdolares - lag_2_Master_mconsumosdolares as delta_2_Master_mconsumosdolares
    , lag(Master_mlimitecompra, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_mlimitecompra
    , Master_mlimitecompra - lag_1_Master_mlimitecompra as delta_1_Master_mlimitecompra
    , lag(Master_mlimitecompra, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Master_mlimitecompra
    , Master_mlimitecompra - lag_2_Master_mlimitecompra as delta_2_Master_mlimitecompra
    , lag(Master_madelantopesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_madelantopesos
    , Master_madelantopesos - lag_1_Master_madelantopesos as delta_1_Master_madelantopesos
    , lag(Master_madelantopesos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Master_madelantopesos
    , Master_madelantopesos - lag_2_Master_madelantopesos as delta_2_Master_madelantopesos
    , lag(Master_madelantodolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_madelantodolares
    , Master_madelantodolares - lag_1_Master_madelantodolares as delta_1_Master_madelantodolares
    , lag(Master_madelantodolares, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Master_madelantodolares
    , Master_madelantodolares - lag_2_Master_madelantodolares as delta_2_Master_madelantodolares
    , lag(Master_fultimo_cierre, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_fultimo_cierre
    , Master_fultimo_cierre - lag_1_Master_fultimo_cierre as delta_1_Master_fultimo_cierre
    , lag(Master_fultimo_cierre, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Master_fultimo_cierre
    , Master_fultimo_cierre - lag_2_Master_fultimo_cierre as delta_2_Master_fultimo_cierre
    , lag(Master_mpagado, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_mpagado
    , Master_mpagado - lag_1_Master_mpagado as delta_1_Master_mpagado
    , lag(Master_mpagado, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Master_mpagado
    , Master_mpagado - lag_2_Master_mpagado as delta_2_Master_mpagado
    , lag(Master_mpagospesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_mpagospesos
    , Master_mpagospesos - lag_1_Master_mpagospesos as delta_1_Master_mpagospesos
    , lag(Master_mpagospesos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Master_mpagospesos
    , Master_mpagospesos - lag_2_Master_mpagospesos as delta_2_Master_mpagospesos
    , lag(Master_mpagosdolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_mpagosdolares
    , Master_mpagosdolares - lag_1_Master_mpagosdolares as delta_1_Master_mpagosdolares
    , lag(Master_mpagosdolares, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Master_mpagosdolares
    , Master_mpagosdolares - lag_2_Master_mpagosdolares as delta_2_Master_mpagosdolares
    , lag(Master_fechaalta, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_fechaalta
    , Master_fechaalta - lag_1_Master_fechaalta as delta_1_Master_fechaalta
    , lag(Master_fechaalta, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Master_fechaalta
    , Master_fechaalta - lag_2_Master_fechaalta as delta_2_Master_fechaalta
    , lag(Master_mconsumototal, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_mconsumototal
    , Master_mconsumototal - lag_1_Master_mconsumototal as delta_1_Master_mconsumototal
    , lag(Master_mconsumototal, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Master_mconsumototal
    , Master_mconsumototal - lag_2_Master_mconsumototal as delta_2_Master_mconsumototal
    , lag(Master_cconsumos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_cconsumos
    , Master_cconsumos - lag_1_Master_cconsumos as delta_1_Master_cconsumos
    , lag(Master_cconsumos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Master_cconsumos
    , Master_cconsumos - lag_2_Master_cconsumos as delta_2_Master_cconsumos
    , lag(Master_cadelantosefectivo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_cadelantosefectivo
    , Master_cadelantosefectivo - lag_1_Master_cadelantosefectivo as delta_1_Master_cadelantosefectivo
    , lag(Master_cadelantosefectivo, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Master_cadelantosefectivo
    , Master_cadelantosefectivo - lag_2_Master_cadelantosefectivo as delta_2_Master_cadelantosefectivo
    , lag(Master_mpagominimo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_mpagominimo
    , Master_mpagominimo - lag_1_Master_mpagominimo as delta_1_Master_mpagominimo
    , lag(Master_mpagominimo, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Master_mpagominimo
    , Master_mpagominimo - lag_2_Master_mpagominimo as delta_2_Master_mpagominimo
    , lag(Visa_delinquency, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_delinquency
    , Visa_delinquency - lag_1_Visa_delinquency as delta_1_Visa_delinquency
    , lag(Visa_delinquency, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Visa_delinquency
    , Visa_delinquency - lag_2_Visa_delinquency as delta_2_Visa_delinquency
    , lag(Visa_status, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_status
    , Visa_status - lag_1_Visa_status as delta_1_Visa_status
    , lag(Visa_status, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Visa_status
    , Visa_status - lag_2_Visa_status as delta_2_Visa_status
    , lag(Visa_mfinanciacion_limite, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mfinanciacion_limite
    , Visa_mfinanciacion_limite - lag_1_Visa_mfinanciacion_limite as delta_1_Visa_mfinanciacion_limite
    , lag(Visa_mfinanciacion_limite, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Visa_mfinanciacion_limite
    , Visa_mfinanciacion_limite - lag_2_Visa_mfinanciacion_limite as delta_2_Visa_mfinanciacion_limite
    , lag(Visa_Fvencimiento, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_Fvencimiento
    , Visa_Fvencimiento - lag_1_Visa_Fvencimiento as delta_1_Visa_Fvencimiento
    , lag(Visa_Fvencimiento, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Visa_Fvencimiento
    , Visa_Fvencimiento - lag_2_Visa_Fvencimiento as delta_2_Visa_Fvencimiento
    , lag(Visa_Finiciomora, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_Finiciomora
    , Visa_Finiciomora - lag_1_Visa_Finiciomora as delta_1_Visa_Finiciomora
    , lag(Visa_Finiciomora, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Visa_Finiciomora
    , Visa_Finiciomora - lag_2_Visa_Finiciomora as delta_2_Visa_Finiciomora
    , lag(Visa_msaldototal, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_msaldototal
    , Visa_msaldototal - lag_1_Visa_msaldototal as delta_1_Visa_msaldototal
    , lag(Visa_msaldototal, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Visa_msaldototal
    , Visa_msaldototal - lag_2_Visa_msaldototal as delta_2_Visa_msaldototal
    , lag(Visa_msaldopesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_msaldopesos
    , Visa_msaldopesos - lag_1_Visa_msaldopesos as delta_1_Visa_msaldopesos
    , lag(Visa_msaldopesos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Visa_msaldopesos
    , Visa_msaldopesos - lag_2_Visa_msaldopesos as delta_2_Visa_msaldopesos
    , lag(Visa_msaldodolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_msaldodolares
    , Visa_msaldodolares - lag_1_Visa_msaldodolares as delta_1_Visa_msaldodolares
    , lag(Visa_msaldodolares, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Visa_msaldodolares
    , Visa_msaldodolares - lag_2_Visa_msaldodolares as delta_2_Visa_msaldodolares
    , lag(Visa_mconsumospesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mconsumospesos
    , Visa_mconsumospesos - lag_1_Visa_mconsumospesos as delta_1_Visa_mconsumospesos
    , lag(Visa_mconsumospesos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Visa_mconsumospesos
    , Visa_mconsumospesos - lag_2_Visa_mconsumospesos as delta_2_Visa_mconsumospesos
    , lag(Visa_mconsumosdolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mconsumosdolares
    , Visa_mconsumosdolares - lag_1_Visa_mconsumosdolares as delta_1_Visa_mconsumosdolares
    , lag(Visa_mconsumosdolares, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Visa_mconsumosdolares
    , Visa_mconsumosdolares - lag_2_Visa_mconsumosdolares as delta_2_Visa_mconsumosdolares
    , lag(Visa_mlimitecompra, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mlimitecompra
    , Visa_mlimitecompra - lag_1_Visa_mlimitecompra as delta_1_Visa_mlimitecompra
    , lag(Visa_mlimitecompra, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Visa_mlimitecompra
    , Visa_mlimitecompra - lag_2_Visa_mlimitecompra as delta_2_Visa_mlimitecompra
    , lag(Visa_madelantopesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_madelantopesos
    , Visa_madelantopesos - lag_1_Visa_madelantopesos as delta_1_Visa_madelantopesos
    , lag(Visa_madelantopesos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Visa_madelantopesos
    , Visa_madelantopesos - lag_2_Visa_madelantopesos as delta_2_Visa_madelantopesos
    , lag(Visa_madelantodolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_madelantodolares
    , Visa_madelantodolares - lag_1_Visa_madelantodolares as delta_1_Visa_madelantodolares
    , lag(Visa_madelantodolares, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Visa_madelantodolares
    , Visa_madelantodolares - lag_2_Visa_madelantodolares as delta_2_Visa_madelantodolares
    , lag(Visa_fultimo_cierre, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_fultimo_cierre
    , Visa_fultimo_cierre - lag_1_Visa_fultimo_cierre as delta_1_Visa_fultimo_cierre
    , lag(Visa_fultimo_cierre, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Visa_fultimo_cierre
    , Visa_fultimo_cierre - lag_2_Visa_fultimo_cierre as delta_2_Visa_fultimo_cierre
    , lag(Visa_mpagado, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mpagado
    , Visa_mpagado - lag_1_Visa_mpagado as delta_1_Visa_mpagado
    , lag(Visa_mpagado, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Visa_mpagado
    , Visa_mpagado - lag_2_Visa_mpagado as delta_2_Visa_mpagado
    , lag(Visa_mpagospesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mpagospesos
    , Visa_mpagospesos - lag_1_Visa_mpagospesos as delta_1_Visa_mpagospesos
    , lag(Visa_mpagospesos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Visa_mpagospesos
    , Visa_mpagospesos - lag_2_Visa_mpagospesos as delta_2_Visa_mpagospesos
    , lag(Visa_mpagosdolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mpagosdolares
    , Visa_mpagosdolares - lag_1_Visa_mpagosdolares as delta_1_Visa_mpagosdolares
    , lag(Visa_mpagosdolares, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Visa_mpagosdolares
    , Visa_mpagosdolares - lag_2_Visa_mpagosdolares as delta_2_Visa_mpagosdolares
    , lag(Visa_fechaalta, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_fechaalta
    , Visa_fechaalta - lag_1_Visa_fechaalta as delta_1_Visa_fechaalta
    , lag(Visa_fechaalta, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Visa_fechaalta
    , Visa_fechaalta - lag_2_Visa_fechaalta as delta_2_Visa_fechaalta
    , lag(Visa_mconsumototal, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mconsumototal
    , Visa_mconsumototal - lag_1_Visa_mconsumototal as delta_1_Visa_mconsumototal
    , lag(Visa_mconsumototal, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Visa_mconsumototal
    , Visa_mconsumototal - lag_2_Visa_mconsumototal as delta_2_Visa_mconsumototal
    , lag(Visa_cconsumos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_cconsumos
    , Visa_cconsumos - lag_1_Visa_cconsumos as delta_1_Visa_cconsumos
    , lag(Visa_cconsumos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Visa_cconsumos
    , Visa_cconsumos - lag_2_Visa_cconsumos as delta_2_Visa_cconsumos
    , lag(Visa_cadelantosefectivo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_cadelantosefectivo
    , Visa_cadelantosefectivo - lag_1_Visa_cadelantosefectivo as delta_1_Visa_cadelantosefectivo
    , lag(Visa_cadelantosefectivo, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Visa_cadelantosefectivo
    , Visa_cadelantosefectivo - lag_2_Visa_cadelantosefectivo as delta_2_Visa_cadelantosefectivo
    , lag(Visa_mpagominimo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mpagominimo
    , Visa_mpagominimo - lag_1_Visa_mpagominimo as delta_1_Visa_mpagominimo
    , lag(Visa_mpagominimo, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Visa_mpagominimo
    , Visa_mpagominimo - lag_2_Visa_mpagominimo as delta_2_Visa_mpagominimo
FROM competencia_01

,Success


In [38]:
%%sql
select 
    *
from competencia_01_lag_delta
limit 10

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,lag_2_Visa_cconsumos,delta_2_Visa_cconsumos,lag_1_Visa_cadelantosefectivo,delta_1_Visa_cadelantosefectivo,lag_2_Visa_cadelantosefectivo,delta_2_Visa_cadelantosefectivo,lag_1_Visa_mpagominimo,delta_1_Visa_mpagominimo,lag_2_Visa_mpagominimo,delta_2_Visa_mpagominimo
0,249328966,202101,1,0,0,55,211,151.73,1186.88,1571.11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,249328966,202102,1,0,0,55,212,-1329.44,330.71,757.15,...,NaN,NaN,0.0,0.0,NaN,NaN,2697.90,293.25,NaN,NaN
2,249328966,202103,1,0,0,55,213,-1933.81,-1397.73,639.68,...,9.0,-3.0,0.0,0.0,0.0,0.0,2991.15,-2017.56,2697.90,-1724.31
3,249328966,202104,1,0,0,55,214,-1278.71,-3019.04,1261.57,...,7.0,-2.0,0.0,0.0,0.0,0.0,973.59,3671.49,2991.15,1653.93
4,249328966,202105,1,0,0,55,215,-1916.50,-5571.29,516.47,...,6.0,1.0,0.0,0.0,0.0,0.0,4645.08,563.04,973.59,4234.53
5,249328966,202106,1,0,0,55,216,-666.07,-5250.61,886.65,...,5.0,1.0,0.0,0.0,0.0,0.0,5208.12,-727.26,4645.08,-164.22
6,249340273,202101,1,0,0,44,273,10026.14,65093.79,3754.24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,249340273,202102,1,0,0,44,274,6707.56,67665.80,720.01,...,NaN,NaN,0.0,0.0,NaN,NaN,1067.43,387.09,NaN,NaN
8,249340273,202103,1,0,0,44,275,7657.67,70243.08,866.50,...,5.0,2.0,0.0,0.0,0.0,0.0,1454.52,-727.26,1067.43,-340.17
9,249340273,202104,1,0,0,44,276,6672.55,72269.17,1015.58,...,11.0,-7.0,0.0,0.0,0.0,0.0,727.26,-23.46,1454.52,-750.72


## Ahora voy con el promedio de cada campo

In [11]:
nuevos_features = ""
for campo in campos:
  nuevos_features += f"\n    ,avg({campo}) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_{campo}"


consulta_sql = f"""
CREATE OR REPLACE TABLE competencia_01_avg AS
select
    numero_de_cliente
    , foto_mes 
    {nuevos_features.strip()}
FROM competencia_01
"""

print(consulta_sql)


CREATE OR REPLACE TABLE competencia_01_avg AS
select
    numero_de_cliente
    , foto_mes 
    ,avg(numero_de_cliente) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_numero_de_cliente
    ,avg(foto_mes) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_foto_mes
    ,avg(active_quarter) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_active_quarter
    ,avg(cliente_vip) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_cliente_vip
    ,avg(internet) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_internet
    ,avg(cliente_edad) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_cliente_edad
    ,avg(cliente_antiguedad) over (partition by numero_de_cl

In [12]:
%%sql
CREATE OR REPLACE TABLE competencia_01_avg AS
select
    numero_de_cliente
    ,foto_mes 
    ,avg(numero_de_cliente) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_numero_de_cliente
    ,avg(foto_mes) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_foto_mes
    ,avg(active_quarter) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_active_quarter
    ,avg(cliente_vip) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_cliente_vip
    ,avg(internet) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_internet
    ,avg(cliente_edad) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_cliente_edad
    ,avg(cliente_antiguedad) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_cliente_antiguedad
    ,avg(mrentabilidad) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mrentabilidad
    ,avg(mrentabilidad_annual) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mrentabilidad_annual
    ,avg(mcomisiones) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mcomisiones
    ,avg(mactivos_margen) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mactivos_margen
    ,avg(mpasivos_margen) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mpasivos_margen
    ,avg(cproductos) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_cproductos
    ,avg(tcuentas) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_tcuentas
    ,avg(ccuenta_corriente) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_ccuenta_corriente
    ,avg(mcuenta_corriente_adicional) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mcuenta_corriente_adicional
    ,avg(mcuenta_corriente) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mcuenta_corriente
    ,avg(ccaja_ahorro) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_ccaja_ahorro
    ,avg(mcaja_ahorro) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mcaja_ahorro
    ,avg(mcaja_ahorro_adicional) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mcaja_ahorro_adicional
    ,avg(mcaja_ahorro_dolares) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mcaja_ahorro_dolares
    ,avg(cdescubierto_preacordado) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_cdescubierto_preacordado
    ,avg(mcuentas_saldo) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mcuentas_saldo
    ,avg(ctarjeta_debito) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_ctarjeta_debito
    ,avg(ctarjeta_debito_transacciones) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_ctarjeta_debito_transacciones
    ,avg(mautoservicio) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mautoservicio
    ,avg(ctarjeta_visa) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_ctarjeta_visa
    ,avg(ctarjeta_visa_transacciones) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_ctarjeta_visa_transacciones
    ,avg(mtarjeta_visa_consumo) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mtarjeta_visa_consumo
    ,avg(ctarjeta_master) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_ctarjeta_master
    ,avg(ctarjeta_master_transacciones) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_ctarjeta_master_transacciones
    ,avg(mtarjeta_master_consumo) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mtarjeta_master_consumo
    ,avg(cprestamos_personales) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_cprestamos_personales
    ,avg(mprestamos_personales) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mprestamos_personales
    ,avg(cprestamos_prendarios) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_cprestamos_prendarios
    ,avg(mprestamos_prendarios) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mprestamos_prendarios
    ,avg(cprestamos_hipotecarios) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_cprestamos_hipotecarios
    ,avg(mprestamos_hipotecarios) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mprestamos_hipotecarios
    ,avg(cplazo_fijo) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_cplazo_fijo
    ,avg(mplazo_fijo_dolares) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mplazo_fijo_dolares
    ,avg(mplazo_fijo_pesos) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mplazo_fijo_pesos
    ,avg(cinversion1) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_cinversion1
    ,avg(minversion1_pesos) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_minversion1_pesos
    ,avg(minversion1_dolares) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_minversion1_dolares
    ,avg(cinversion2) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_cinversion2
    ,avg(minversion2) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_minversion2
    ,avg(cseguro_vida) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_cseguro_vida
    ,avg(cseguro_auto) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_cseguro_auto
    ,avg(cseguro_vivienda) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_cseguro_vivienda
    ,avg(cseguro_accidentes_personales) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_cseguro_accidentes_personales
    ,avg(ccaja_seguridad) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_ccaja_seguridad
    ,avg(cpayroll_trx) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_cpayroll_trx
    ,avg(mpayroll) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mpayroll
    ,avg(mpayroll2) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mpayroll2
    ,avg(cpayroll2_trx) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_cpayroll2_trx
    ,avg(ccuenta_debitos_automaticos) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_ccuenta_debitos_automaticos
    ,avg(mcuenta_debitos_automaticos) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mcuenta_debitos_automaticos
    ,avg(ctarjeta_visa_debitos_automaticos) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_ctarjeta_visa_debitos_automaticos
    ,avg(mttarjeta_visa_debitos_automaticos) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mttarjeta_visa_debitos_automaticos
    ,avg(ctarjeta_master_debitos_automaticos) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_ctarjeta_master_debitos_automaticos
    ,avg(mttarjeta_master_debitos_automaticos) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mttarjeta_master_debitos_automaticos
    ,avg(cpagodeservicios) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_cpagodeservicios
    ,avg(mpagodeservicios) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mpagodeservicios
    ,avg(cpagomiscuentas) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_cpagomiscuentas
    ,avg(mpagomiscuentas) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mpagomiscuentas
    ,avg(ccajeros_propios_descuentos) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_ccajeros_propios_descuentos
    ,avg(mcajeros_propios_descuentos) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mcajeros_propios_descuentos
    ,avg(ctarjeta_visa_descuentos) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_ctarjeta_visa_descuentos
    ,avg(mtarjeta_visa_descuentos) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mtarjeta_visa_descuentos
    ,avg(ctarjeta_master_descuentos) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_ctarjeta_master_descuentos
    ,avg(mtarjeta_master_descuentos) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mtarjeta_master_descuentos
    ,avg(ccomisiones_mantenimiento) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_ccomisiones_mantenimiento
    ,avg(mcomisiones_mantenimiento) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mcomisiones_mantenimiento
    ,avg(ccomisiones_otras) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_ccomisiones_otras
    ,avg(mcomisiones_otras) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mcomisiones_otras
    ,avg(cforex) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_cforex
    ,avg(cforex_buy) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_cforex_buy
    ,avg(mforex_buy) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mforex_buy
    ,avg(cforex_sell) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_cforex_sell
    ,avg(mforex_sell) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mforex_sell
    ,avg(ctransferencias_recibidas) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_ctransferencias_recibidas
    ,avg(mtransferencias_recibidas) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mtransferencias_recibidas
    ,avg(ctransferencias_emitidas) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_ctransferencias_emitidas
    ,avg(mtransferencias_emitidas) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mtransferencias_emitidas
    ,avg(cextraccion_autoservicio) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_cextraccion_autoservicio
    ,avg(mextraccion_autoservicio) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mextraccion_autoservicio
    ,avg(ccheques_depositados) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_ccheques_depositados
    ,avg(mcheques_depositados) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mcheques_depositados
    ,avg(ccheques_emitidos) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_ccheques_emitidos
    ,avg(mcheques_emitidos) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mcheques_emitidos
    ,avg(ccheques_depositados_rechazados) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_ccheques_depositados_rechazados
    ,avg(mcheques_depositados_rechazados) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mcheques_depositados_rechazados
    ,avg(ccheques_emitidos_rechazados) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_ccheques_emitidos_rechazados
    ,avg(mcheques_emitidos_rechazados) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mcheques_emitidos_rechazados
    ,avg(tcallcenter) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_tcallcenter
    ,avg(ccallcenter_transacciones) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_ccallcenter_transacciones
    ,avg(thomebanking) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_thomebanking
    ,avg(chomebanking_transacciones) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_chomebanking_transacciones
    ,avg(ccajas_transacciones) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_ccajas_transacciones
    ,avg(ccajas_consultas) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_ccajas_consultas
    ,avg(ccajas_depositos) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_ccajas_depositos
    ,avg(ccajas_extracciones) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_ccajas_extracciones
    ,avg(ccajas_otras) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_ccajas_otras
    ,avg(catm_trx) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_catm_trx
    ,avg(matm) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_matm
    ,avg(catm_trx_other) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_catm_trx_other
    ,avg(matm_other) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_matm_other
    ,avg(ctrx_quarter) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_ctrx_quarter
    ,avg(tmobile_app) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_tmobile_app
    ,avg(cmobile_app_trx) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_cmobile_app_trx
    ,avg(Master_delinquency) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_Master_delinquency
    ,avg(Master_status) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_Master_status
    ,avg(Master_mfinanciacion_limite) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_Master_mfinanciacion_limite
    ,avg(Master_Fvencimiento) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_Master_Fvencimiento
    ,avg(Master_Finiciomora) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_Master_Finiciomora
    ,avg(Master_msaldototal) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_Master_msaldototal
    ,avg(Master_msaldopesos) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_Master_msaldopesos
    ,avg(Master_msaldodolares) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_Master_msaldodolares
    ,avg(Master_mconsumospesos) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_Master_mconsumospesos
    ,avg(Master_mconsumosdolares) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_Master_mconsumosdolares
    ,avg(Master_mlimitecompra) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_Master_mlimitecompra
    ,avg(Master_madelantopesos) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_Master_madelantopesos
    ,avg(Master_madelantodolares) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_Master_madelantodolares
    ,avg(Master_fultimo_cierre) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_Master_fultimo_cierre
    ,avg(Master_mpagado) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_Master_mpagado
    ,avg(Master_mpagospesos) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_Master_mpagospesos
    ,avg(Master_mpagosdolares) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_Master_mpagosdolares
    ,avg(Master_fechaalta) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_Master_fechaalta
    ,avg(Master_mconsumototal) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_Master_mconsumototal
    ,avg(Master_cconsumos) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_Master_cconsumos
    ,avg(Master_cadelantosefectivo) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_Master_cadelantosefectivo
    ,avg(Master_mpagominimo) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_Master_mpagominimo
    ,avg(Visa_delinquency) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_Visa_delinquency
    ,avg(Visa_status) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_Visa_status
    ,avg(Visa_mfinanciacion_limite) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_Visa_mfinanciacion_limite
    ,avg(Visa_Fvencimiento) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_Visa_Fvencimiento
    ,avg(Visa_Finiciomora) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_Visa_Finiciomora
    ,avg(Visa_msaldototal) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_Visa_msaldototal
    ,avg(Visa_msaldopesos) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_Visa_msaldopesos
    ,avg(Visa_msaldodolares) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_Visa_msaldodolares
    ,avg(Visa_mconsumospesos) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_Visa_mconsumospesos
    ,avg(Visa_mconsumosdolares) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_Visa_mconsumosdolares
    ,avg(Visa_mlimitecompra) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_Visa_mlimitecompra
    ,avg(Visa_madelantopesos) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_Visa_madelantopesos
    ,avg(Visa_madelantodolares) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_Visa_madelantodolares
    ,avg(Visa_fultimo_cierre) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_Visa_fultimo_cierre
    ,avg(Visa_mpagado) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_Visa_mpagado
    ,avg(Visa_mpagospesos) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_Visa_mpagospesos
    ,avg(Visa_mpagosdolares) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_Visa_mpagosdolares
    ,avg(Visa_fechaalta) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_Visa_fechaalta
    ,avg(Visa_mconsumototal) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_Visa_mconsumototal
    ,avg(Visa_cconsumos) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_Visa_cconsumos
    ,avg(Visa_cadelantosefectivo) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_Visa_cadelantosefectivo
    ,avg(Visa_mpagominimo) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_Visa_mpagominimo
FROM competencia_01


,Success


In [13]:
%%sql
select 
    *
from competencia_01_avg

,numero_de_cliente,foto_mes,avg_3_numero_de_cliente,avg_3_foto_mes,avg_3_active_quarter,avg_3_cliente_vip,avg_3_internet,avg_3_cliente_edad,avg_3_cliente_antiguedad,avg_3_mrentabilidad,...,avg_3_Visa_madelantodolares,avg_3_Visa_fultimo_cierre,avg_3_Visa_mpagado,avg_3_Visa_mpagospesos,avg_3_Visa_mpagosdolares,avg_3_Visa_fechaalta,avg_3_Visa_mconsumototal,avg_3_Visa_cconsumos,avg_3_Visa_cadelantosefectivo,avg_3_Visa_mpagominimo
0,249236712,202101,2.492367e+08,202101.0,1.0,0.0,0.0,44.0,25.0,2074.770000,...,0.0,4.000000,0.0,-4986.790,0.0,741.0,0.0,0.0,0.0,2005.8300
1,249236712,202102,2.492367e+08,202101.5,1.0,0.0,0.0,44.0,25.5,1765.890000,...,0.0,0.500000,0.0,-4986.790,0.0,755.0,0.0,0.0,0.0,1929.5850
2,249236712,202103,2.492367e+08,202102.0,1.0,0.0,0.0,44.0,26.0,1884.706667,...,0.0,0.666667,0.0,-4543.690,0.0,770.0,0.0,0.0,0.0,2127.0400
3,249236712,202104,2.492367e+08,202102.5,1.0,0.0,0.0,44.0,26.5,1897.892500,...,0.0,1.000000,0.0,-4489.865,0.0,785.0,0.0,0.0,0.0,2302.0125
4,249236712,202105,2.492367e+08,202103.5,1.0,0.0,0.0,44.0,27.5,3289.520000,...,0.0,1.250000,0.0,-4637.085,0.0,815.0,0.0,0.0,0.0,2507.2875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
981941,1595193701,202105,1.595194e+09,202105.0,0.0,0.0,0.0,51.0,1.0,0.000000,...,NaN,NaN,0.0,NaN,NaN,1.0,NaN,NaN,NaN,0.0000
981942,1595193701,202106,1.595194e+09,202105.5,0.5,0.0,0.0,51.0,1.5,588.475000,...,NaN,0.000000,0.0,NaN,NaN,16.0,NaN,NaN,NaN,0.0000
981943,1598206246,202106,1.598206e+09,202106.0,1.0,0.0,0.0,31.0,1.0,9.090000,...,NaN,0.000000,0.0,NaN,NaN,7.0,NaN,NaN,NaN,0.0000
981944,1598444941,202106,1.598445e+09,202106.0,0.0,0.0,0.0,27.0,1.0,0.000000,...,NaN,NaN,0.0,NaN,NaN,15.0,NaN,NaN,NaN,0.0000


## reg_slope

In [68]:
nuevos_features = ""
for campo in campos:
  nuevos_features +=  f"\n    ,regr_slope({campo}, cliente_antiguedad) over ventana_3 as {campo}_slope_3"


consulta_sql = f"""
CREATE OR REPLACE TABLE competencia_01_regr_slope AS
select
    numero_de_cliente
    , foto_mes 
    {nuevos_features.strip()}
from competencia_01
window ventana_3 as (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row)
"""

print(consulta_sql)


CREATE OR REPLACE TABLE competencia_01_regr_slope AS
select
    numero_de_cliente
    , foto_mes 
    ,regr_slope(numero_de_cliente, cliente_antiguedad) over ventana_3 as numero_de_cliente_slope_3
    ,regr_slope(foto_mes, cliente_antiguedad) over ventana_3 as foto_mes_slope_3
    ,regr_slope(active_quarter, cliente_antiguedad) over ventana_3 as active_quarter_slope_3
    ,regr_slope(cliente_vip, cliente_antiguedad) over ventana_3 as cliente_vip_slope_3
    ,regr_slope(internet, cliente_antiguedad) over ventana_3 as internet_slope_3
    ,regr_slope(cliente_edad, cliente_antiguedad) over ventana_3 as cliente_edad_slope_3
    ,regr_slope(cliente_antiguedad, cliente_antiguedad) over ventana_3 as cliente_antiguedad_slope_3
    ,regr_slope(mrentabilidad, cliente_antiguedad) over ventana_3 as mrentabilidad_slope_3
    ,regr_slope(mrentabilidad_annual, cliente_antiguedad) over ventana_3 as mrentabilidad_annual_slope_3
    ,regr_slope(mcomisiones, cliente_antiguedad) over ventana_3 as mcomisi

In [13]:
%%sql
CREATE OR REPLACE TABLE competencia_01_regr_slope AS
select
    numero_de_cliente
    , foto_mes 
    ,regr_slope(numero_de_cliente, cliente_antiguedad) over ventana_3 as numero_de_cliente_slope_3
    ,regr_slope(foto_mes, cliente_antiguedad) over ventana_3 as foto_mes_slope_3
    ,regr_slope(active_quarter, cliente_antiguedad) over ventana_3 as active_quarter_slope_3
    ,regr_slope(cliente_vip, cliente_antiguedad) over ventana_3 as cliente_vip_slope_3
    ,regr_slope(internet, cliente_antiguedad) over ventana_3 as internet_slope_3
    ,regr_slope(cliente_edad, cliente_antiguedad) over ventana_3 as cliente_edad_slope_3
    ,regr_slope(cliente_antiguedad, cliente_antiguedad) over ventana_3 as cliente_antiguedad_slope_3
    ,regr_slope(mrentabilidad, cliente_antiguedad) over ventana_3 as mrentabilidad_slope_3
    ,regr_slope(mrentabilidad_annual, cliente_antiguedad) over ventana_3 as mrentabilidad_annual_slope_3
    ,regr_slope(mcomisiones, cliente_antiguedad) over ventana_3 as mcomisiones_slope_3
    ,regr_slope(mactivos_margen, cliente_antiguedad) over ventana_3 as mactivos_margen_slope_3
    ,regr_slope(mpasivos_margen, cliente_antiguedad) over ventana_3 as mpasivos_margen_slope_3
    ,regr_slope(cproductos, cliente_antiguedad) over ventana_3 as cproductos_slope_3
    ,regr_slope(tcuentas, cliente_antiguedad) over ventana_3 as tcuentas_slope_3
    ,regr_slope(ccuenta_corriente, cliente_antiguedad) over ventana_3 as ccuenta_corriente_slope_3
    ,regr_slope(mcuenta_corriente_adicional, cliente_antiguedad) over ventana_3 as mcuenta_corriente_adicional_slope_3
    ,regr_slope(mcuenta_corriente, cliente_antiguedad) over ventana_3 as mcuenta_corriente_slope_3
    ,regr_slope(ccaja_ahorro, cliente_antiguedad) over ventana_3 as ccaja_ahorro_slope_3
    ,regr_slope(mcaja_ahorro, cliente_antiguedad) over ventana_3 as mcaja_ahorro_slope_3
    ,regr_slope(mcaja_ahorro_adicional, cliente_antiguedad) over ventana_3 as mcaja_ahorro_adicional_slope_3
    ,regr_slope(mcaja_ahorro_dolares, cliente_antiguedad) over ventana_3 as mcaja_ahorro_dolares_slope_3
    ,regr_slope(cdescubierto_preacordado, cliente_antiguedad) over ventana_3 as cdescubierto_preacordado_slope_3
    ,regr_slope(mcuentas_saldo, cliente_antiguedad) over ventana_3 as mcuentas_saldo_slope_3
    ,regr_slope(ctarjeta_debito, cliente_antiguedad) over ventana_3 as ctarjeta_debito_slope_3
    ,regr_slope(ctarjeta_debito_transacciones, cliente_antiguedad) over ventana_3 as ctarjeta_debito_transacciones_slope_3
    ,regr_slope(mautoservicio, cliente_antiguedad) over ventana_3 as mautoservicio_slope_3
    ,regr_slope(ctarjeta_visa, cliente_antiguedad) over ventana_3 as ctarjeta_visa_slope_3
    ,regr_slope(ctarjeta_visa_transacciones, cliente_antiguedad) over ventana_3 as ctarjeta_visa_transacciones_slope_3
    ,regr_slope(mtarjeta_visa_consumo, cliente_antiguedad) over ventana_3 as mtarjeta_visa_consumo_slope_3
    ,regr_slope(ctarjeta_master, cliente_antiguedad) over ventana_3 as ctarjeta_master_slope_3
    ,regr_slope(ctarjeta_master_transacciones, cliente_antiguedad) over ventana_3 as ctarjeta_master_transacciones_slope_3
    ,regr_slope(mtarjeta_master_consumo, cliente_antiguedad) over ventana_3 as mtarjeta_master_consumo_slope_3
    ,regr_slope(cprestamos_personales, cliente_antiguedad) over ventana_3 as cprestamos_personales_slope_3
    ,regr_slope(mprestamos_personales, cliente_antiguedad) over ventana_3 as mprestamos_personales_slope_3
    ,regr_slope(cprestamos_prendarios, cliente_antiguedad) over ventana_3 as cprestamos_prendarios_slope_3
    ,regr_slope(mprestamos_prendarios, cliente_antiguedad) over ventana_3 as mprestamos_prendarios_slope_3
    ,regr_slope(cprestamos_hipotecarios, cliente_antiguedad) over ventana_3 as cprestamos_hipotecarios_slope_3
    ,regr_slope(mprestamos_hipotecarios, cliente_antiguedad) over ventana_3 as mprestamos_hipotecarios_slope_3
    ,regr_slope(cplazo_fijo, cliente_antiguedad) over ventana_3 as cplazo_fijo_slope_3
    ,regr_slope(mplazo_fijo_dolares, cliente_antiguedad) over ventana_3 as mplazo_fijo_dolares_slope_3
    ,regr_slope(mplazo_fijo_pesos, cliente_antiguedad) over ventana_3 as mplazo_fijo_pesos_slope_3
    ,regr_slope(cinversion1, cliente_antiguedad) over ventana_3 as cinversion1_slope_3
    ,regr_slope(minversion1_pesos, cliente_antiguedad) over ventana_3 as minversion1_pesos_slope_3
    ,regr_slope(minversion1_dolares, cliente_antiguedad) over ventana_3 as minversion1_dolares_slope_3
    ,regr_slope(cinversion2, cliente_antiguedad) over ventana_3 as cinversion2_slope_3
    ,regr_slope(minversion2, cliente_antiguedad) over ventana_3 as minversion2_slope_3
    ,regr_slope(cseguro_vida, cliente_antiguedad) over ventana_3 as cseguro_vida_slope_3
    ,regr_slope(cseguro_auto, cliente_antiguedad) over ventana_3 as cseguro_auto_slope_3
    ,regr_slope(cseguro_vivienda, cliente_antiguedad) over ventana_3 as cseguro_vivienda_slope_3
    ,regr_slope(cseguro_accidentes_personales, cliente_antiguedad) over ventana_3 as cseguro_accidentes_personales_slope_3
    ,regr_slope(ccaja_seguridad, cliente_antiguedad) over ventana_3 as ccaja_seguridad_slope_3
    ,regr_slope(cpayroll_trx, cliente_antiguedad) over ventana_3 as cpayroll_trx_slope_3
    ,regr_slope(mpayroll, cliente_antiguedad) over ventana_3 as mpayroll_slope_3
    ,regr_slope(mpayroll2, cliente_antiguedad) over ventana_3 as mpayroll2_slope_3
    ,regr_slope(cpayroll2_trx, cliente_antiguedad) over ventana_3 as cpayroll2_trx_slope_3
    ,regr_slope(ccuenta_debitos_automaticos, cliente_antiguedad) over ventana_3 as ccuenta_debitos_automaticos_slope_3
    ,regr_slope(mcuenta_debitos_automaticos, cliente_antiguedad) over ventana_3 as mcuenta_debitos_automaticos_slope_3
    ,regr_slope(ctarjeta_visa_debitos_automaticos, cliente_antiguedad) over ventana_3 as ctarjeta_visa_debitos_automaticos_slope_3
    ,regr_slope(mttarjeta_visa_debitos_automaticos, cliente_antiguedad) over ventana_3 as mttarjeta_visa_debitos_automaticos_slope_3
    ,regr_slope(ctarjeta_master_debitos_automaticos, cliente_antiguedad) over ventana_3 as ctarjeta_master_debitos_automaticos_slope_3
    ,regr_slope(mttarjeta_master_debitos_automaticos, cliente_antiguedad) over ventana_3 as mttarjeta_master_debitos_automaticos_slope_3
    ,regr_slope(cpagodeservicios, cliente_antiguedad) over ventana_3 as cpagodeservicios_slope_3
    ,regr_slope(mpagodeservicios, cliente_antiguedad) over ventana_3 as mpagodeservicios_slope_3
    ,regr_slope(cpagomiscuentas, cliente_antiguedad) over ventana_3 as cpagomiscuentas_slope_3
    ,regr_slope(mpagomiscuentas, cliente_antiguedad) over ventana_3 as mpagomiscuentas_slope_3
    ,regr_slope(ccajeros_propios_descuentos, cliente_antiguedad) over ventana_3 as ccajeros_propios_descuentos_slope_3
    ,regr_slope(mcajeros_propios_descuentos, cliente_antiguedad) over ventana_3 as mcajeros_propios_descuentos_slope_3
    ,regr_slope(ctarjeta_visa_descuentos, cliente_antiguedad) over ventana_3 as ctarjeta_visa_descuentos_slope_3
    ,regr_slope(mtarjeta_visa_descuentos, cliente_antiguedad) over ventana_3 as mtarjeta_visa_descuentos_slope_3
    ,regr_slope(ctarjeta_master_descuentos, cliente_antiguedad) over ventana_3 as ctarjeta_master_descuentos_slope_3
    ,regr_slope(mtarjeta_master_descuentos, cliente_antiguedad) over ventana_3 as mtarjeta_master_descuentos_slope_3
    ,regr_slope(ccomisiones_mantenimiento, cliente_antiguedad) over ventana_3 as ccomisiones_mantenimiento_slope_3
    ,regr_slope(mcomisiones_mantenimiento, cliente_antiguedad) over ventana_3 as mcomisiones_mantenimiento_slope_3
    ,regr_slope(ccomisiones_otras, cliente_antiguedad) over ventana_3 as ccomisiones_otras_slope_3
    ,regr_slope(mcomisiones_otras, cliente_antiguedad) over ventana_3 as mcomisiones_otras_slope_3
    ,regr_slope(cforex, cliente_antiguedad) over ventana_3 as cforex_slope_3
    ,regr_slope(cforex_buy, cliente_antiguedad) over ventana_3 as cforex_buy_slope_3
    ,regr_slope(mforex_buy, cliente_antiguedad) over ventana_3 as mforex_buy_slope_3
    ,regr_slope(cforex_sell, cliente_antiguedad) over ventana_3 as cforex_sell_slope_3
    ,regr_slope(mforex_sell, cliente_antiguedad) over ventana_3 as mforex_sell_slope_3
    ,regr_slope(ctransferencias_recibidas, cliente_antiguedad) over ventana_3 as ctransferencias_recibidas_slope_3
    ,regr_slope(mtransferencias_recibidas, cliente_antiguedad) over ventana_3 as mtransferencias_recibidas_slope_3
    ,regr_slope(ctransferencias_emitidas, cliente_antiguedad) over ventana_3 as ctransferencias_emitidas_slope_3
    ,regr_slope(mtransferencias_emitidas, cliente_antiguedad) over ventana_3 as mtransferencias_emitidas_slope_3
    ,regr_slope(cextraccion_autoservicio, cliente_antiguedad) over ventana_3 as cextraccion_autoservicio_slope_3
    ,regr_slope(mextraccion_autoservicio, cliente_antiguedad) over ventana_3 as mextraccion_autoservicio_slope_3
    ,regr_slope(ccheques_depositados, cliente_antiguedad) over ventana_3 as ccheques_depositados_slope_3
    ,regr_slope(mcheques_depositados, cliente_antiguedad) over ventana_3 as mcheques_depositados_slope_3
    ,regr_slope(ccheques_emitidos, cliente_antiguedad) over ventana_3 as ccheques_emitidos_slope_3
    ,regr_slope(mcheques_emitidos, cliente_antiguedad) over ventana_3 as mcheques_emitidos_slope_3
    ,regr_slope(ccheques_depositados_rechazados, cliente_antiguedad) over ventana_3 as ccheques_depositados_rechazados_slope_3
    ,regr_slope(mcheques_depositados_rechazados, cliente_antiguedad) over ventana_3 as mcheques_depositados_rechazados_slope_3
    ,regr_slope(ccheques_emitidos_rechazados, cliente_antiguedad) over ventana_3 as ccheques_emitidos_rechazados_slope_3
    ,regr_slope(mcheques_emitidos_rechazados, cliente_antiguedad) over ventana_3 as mcheques_emitidos_rechazados_slope_3
    ,regr_slope(tcallcenter, cliente_antiguedad) over ventana_3 as tcallcenter_slope_3
    ,regr_slope(ccallcenter_transacciones, cliente_antiguedad) over ventana_3 as ccallcenter_transacciones_slope_3
    ,regr_slope(thomebanking, cliente_antiguedad) over ventana_3 as thomebanking_slope_3
    ,regr_slope(chomebanking_transacciones, cliente_antiguedad) over ventana_3 as chomebanking_transacciones_slope_3
    ,regr_slope(ccajas_transacciones, cliente_antiguedad) over ventana_3 as ccajas_transacciones_slope_3
    ,regr_slope(ccajas_consultas, cliente_antiguedad) over ventana_3 as ccajas_consultas_slope_3
    ,regr_slope(ccajas_depositos, cliente_antiguedad) over ventana_3 as ccajas_depositos_slope_3
    ,regr_slope(ccajas_extracciones, cliente_antiguedad) over ventana_3 as ccajas_extracciones_slope_3
    ,regr_slope(ccajas_otras, cliente_antiguedad) over ventana_3 as ccajas_otras_slope_3
    ,regr_slope(catm_trx, cliente_antiguedad) over ventana_3 as catm_trx_slope_3
    ,regr_slope(matm, cliente_antiguedad) over ventana_3 as matm_slope_3
    ,regr_slope(catm_trx_other, cliente_antiguedad) over ventana_3 as catm_trx_other_slope_3
    ,regr_slope(matm_other, cliente_antiguedad) over ventana_3 as matm_other_slope_3
    ,regr_slope(ctrx_quarter, cliente_antiguedad) over ventana_3 as ctrx_quarter_slope_3
    ,regr_slope(tmobile_app, cliente_antiguedad) over ventana_3 as tmobile_app_slope_3
    ,regr_slope(cmobile_app_trx, cliente_antiguedad) over ventana_3 as cmobile_app_trx_slope_3
    ,regr_slope(Master_delinquency, cliente_antiguedad) over ventana_3 as Master_delinquency_slope_3
    ,regr_slope(Master_status, cliente_antiguedad) over ventana_3 as Master_status_slope_3
    ,regr_slope(Master_mfinanciacion_limite, cliente_antiguedad) over ventana_3 as Master_mfinanciacion_limite_slope_3
    ,regr_slope(Master_Fvencimiento, cliente_antiguedad) over ventana_3 as Master_Fvencimiento_slope_3
    ,regr_slope(Master_Finiciomora, cliente_antiguedad) over ventana_3 as Master_Finiciomora_slope_3
    ,regr_slope(Master_msaldototal, cliente_antiguedad) over ventana_3 as Master_msaldototal_slope_3
    ,regr_slope(Master_msaldopesos, cliente_antiguedad) over ventana_3 as Master_msaldopesos_slope_3
    ,regr_slope(Master_msaldodolares, cliente_antiguedad) over ventana_3 as Master_msaldodolares_slope_3
    ,regr_slope(Master_mconsumospesos, cliente_antiguedad) over ventana_3 as Master_mconsumospesos_slope_3
    ,regr_slope(Master_mconsumosdolares, cliente_antiguedad) over ventana_3 as Master_mconsumosdolares_slope_3
    ,regr_slope(Master_mlimitecompra, cliente_antiguedad) over ventana_3 as Master_mlimitecompra_slope_3
    ,regr_slope(Master_madelantopesos, cliente_antiguedad) over ventana_3 as Master_madelantopesos_slope_3
    ,regr_slope(Master_madelantodolares, cliente_antiguedad) over ventana_3 as Master_madelantodolares_slope_3
    ,regr_slope(Master_fultimo_cierre, cliente_antiguedad) over ventana_3 as Master_fultimo_cierre_slope_3
    ,regr_slope(Master_mpagado, cliente_antiguedad) over ventana_3 as Master_mpagado_slope_3
    ,regr_slope(Master_mpagospesos, cliente_antiguedad) over ventana_3 as Master_mpagospesos_slope_3
    ,regr_slope(Master_mpagosdolares, cliente_antiguedad) over ventana_3 as Master_mpagosdolares_slope_3
    ,regr_slope(Master_fechaalta, cliente_antiguedad) over ventana_3 as Master_fechaalta_slope_3
    ,regr_slope(Master_mconsumototal, cliente_antiguedad) over ventana_3 as Master_mconsumototal_slope_3
    ,regr_slope(Master_cconsumos, cliente_antiguedad) over ventana_3 as Master_cconsumos_slope_3
    ,regr_slope(Master_cadelantosefectivo, cliente_antiguedad) over ventana_3 as Master_cadelantosefectivo_slope_3
    ,regr_slope(Master_mpagominimo, cliente_antiguedad) over ventana_3 as Master_mpagominimo_slope_3
    ,regr_slope(Visa_delinquency, cliente_antiguedad) over ventana_3 as Visa_delinquency_slope_3
    ,regr_slope(Visa_status, cliente_antiguedad) over ventana_3 as Visa_status_slope_3
    ,regr_slope(Visa_mfinanciacion_limite, cliente_antiguedad) over ventana_3 as Visa_mfinanciacion_limite_slope_3
    ,regr_slope(Visa_Fvencimiento, cliente_antiguedad) over ventana_3 as Visa_Fvencimiento_slope_3
    ,regr_slope(Visa_Finiciomora, cliente_antiguedad) over ventana_3 as Visa_Finiciomora_slope_3
    ,regr_slope(Visa_msaldototal, cliente_antiguedad) over ventana_3 as Visa_msaldototal_slope_3
    ,regr_slope(Visa_msaldopesos, cliente_antiguedad) over ventana_3 as Visa_msaldopesos_slope_3
    ,regr_slope(Visa_msaldodolares, cliente_antiguedad) over ventana_3 as Visa_msaldodolares_slope_3
    ,regr_slope(Visa_mconsumospesos, cliente_antiguedad) over ventana_3 as Visa_mconsumospesos_slope_3
    ,regr_slope(Visa_mconsumosdolares, cliente_antiguedad) over ventana_3 as Visa_mconsumosdolares_slope_3
    ,regr_slope(Visa_mlimitecompra, cliente_antiguedad) over ventana_3 as Visa_mlimitecompra_slope_3
    ,regr_slope(Visa_madelantopesos, cliente_antiguedad) over ventana_3 as Visa_madelantopesos_slope_3
    ,regr_slope(Visa_madelantodolares, cliente_antiguedad) over ventana_3 as Visa_madelantodolares_slope_3
    ,regr_slope(Visa_fultimo_cierre, cliente_antiguedad) over ventana_3 as Visa_fultimo_cierre_slope_3
    ,regr_slope(Visa_mpagado, cliente_antiguedad) over ventana_3 as Visa_mpagado_slope_3
    ,regr_slope(Visa_mpagospesos, cliente_antiguedad) over ventana_3 as Visa_mpagospesos_slope_3
    ,regr_slope(Visa_mpagosdolares, cliente_antiguedad) over ventana_3 as Visa_mpagosdolares_slope_3
    ,regr_slope(Visa_fechaalta, cliente_antiguedad) over ventana_3 as Visa_fechaalta_slope_3
    ,regr_slope(Visa_mconsumototal, cliente_antiguedad) over ventana_3 as Visa_mconsumototal_slope_3
    ,regr_slope(Visa_cconsumos, cliente_antiguedad) over ventana_3 as Visa_cconsumos_slope_3
    ,regr_slope(Visa_cadelantosefectivo, cliente_antiguedad) over ventana_3 as Visa_cadelantosefectivo_slope_3
    ,regr_slope(Visa_mpagominimo, cliente_antiguedad) over ventana_3 as Visa_mpagominimo_slope_3
from competencia_01
window ventana_3 as (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row)

,Success


In [75]:
%%sql
select 
    *
from competencia_01_regr_slope

,numero_de_cliente,foto_mes,numero_de_cliente_slope_3,foto_mes_slope_3,active_quarter_slope_3,cliente_vip_slope_3,internet_slope_3,cliente_edad_slope_3,cliente_antiguedad_slope_3,mrentabilidad_slope_3,...,Visa_madelantodolares_slope_3,Visa_fultimo_cierre_slope_3,Visa_mpagado_slope_3,Visa_mpagospesos_slope_3,Visa_mpagosdolares_slope_3,Visa_fechaalta_slope_3,Visa_mconsumototal_slope_3,Visa_cconsumos_slope_3,Visa_cadelantosefectivo_slope_3,Visa_mpagominimo_slope_3
0,249246268,202101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,249246268,202102,0.0,1.0,0.0,0.0,0.0,0.0,1.0,18.05,...,0.0,-7.0,-210.33,21.190,0.0,28.0,0.000,0.0,0.0,1.173000e+01
2,249301400,202101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,249301400,202102,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1453.09,...,0.0,-7.0,-29341.29,-13631.180,0.0,28.0,-7371.250,-3.0,0.0,-7.272600e+02
4,249301400,202103,0.0,1.0,0.0,0.0,0.5,0.0,1.0,1102.80,...,0.0,-2.0,-11266.53,7855.055,0.0,29.5,-6200.245,-2.5,0.0,-1.705303e-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
981941,1597191683,202106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
981942,1597512564,202106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
981943,1598076801,202106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
981944,1598107578,202106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
### hacer de todos no se puede

campos = [
    "mcuentas_saldo",
    "cpayroll_trx",
    "ctrx_quarter",
    "mcaja_ahorro",
    "mprestamos_personales",
    "mtarjeta_visa_consumo",
    "mpasivos_margen",
    "mcomisiones_mantenimiento",
    "cproductos"
]

In [31]:
nuevos_features = ""
for campo in campos:
  nuevos_features +=  f"\n    ,percent_rank() OVER (ORDER BY {campo}) as percent_rank_{campo}"


consulta_sql = f"""
CREATE OR REPLACE TABLE competencia_01_percent_rank AS
select
    numero_de_cliente
    , foto_mes 
    {nuevos_features.strip()}
from competencia_01

"""

print(consulta_sql)


CREATE OR REPLACE TABLE competencia_01_percent_rank AS
select
    numero_de_cliente
    , foto_mes 
    ,percent_rank() OVER (ORDER BY mcuentas_saldo) as percent_rank_mcuentas_saldo
    ,percent_rank() OVER (ORDER BY cpayroll_trx) as percent_rank_cpayroll_trx
    ,percent_rank() OVER (ORDER BY ctrx_quarter) as percent_rank_ctrx_quarter
    ,percent_rank() OVER (ORDER BY mcaja_ahorro) as percent_rank_mcaja_ahorro
    ,percent_rank() OVER (ORDER BY mprestamos_personales) as percent_rank_mprestamos_personales
    ,percent_rank() OVER (ORDER BY mtarjeta_visa_consumo) as percent_rank_mtarjeta_visa_consumo
    ,percent_rank() OVER (ORDER BY mpasivos_margen) as percent_rank_mpasivos_margen
    ,percent_rank() OVER (ORDER BY mcomisiones_mantenimiento) as percent_rank_mcomisiones_mantenimiento
    ,percent_rank() OVER (ORDER BY cproductos) as percent_rank_cproductos
from competencia_01




In [14]:
%%sql
CREATE OR REPLACE TABLE competencia_01_percent_rank AS
select
    numero_de_cliente
    , foto_mes 
    ,percent_rank() OVER (ORDER BY mcuentas_saldo) as percent_rank_mcuentas_saldo
    ,percent_rank() OVER (ORDER BY cpayroll_trx) as percent_rank_cpayroll_trx
    ,percent_rank() OVER (ORDER BY ctrx_quarter) as percent_rank_ctrx_quarter
    ,percent_rank() OVER (ORDER BY mcaja_ahorro) as percent_rank_mcaja_ahorro
    ,percent_rank() OVER (ORDER BY mprestamos_personales) as percent_rank_mprestamos_personales
    ,percent_rank() OVER (ORDER BY mtarjeta_visa_consumo) as percent_rank_mtarjeta_visa_consumo
    ,percent_rank() OVER (ORDER BY mpasivos_margen) as percent_rank_mpasivos_margen
    ,percent_rank() OVER (ORDER BY mcomisiones_mantenimiento) as percent_rank_mcomisiones_mantenimiento
    ,percent_rank() OVER (ORDER BY cproductos) as percent_rank_cproductos
from competencia_01

,Success


In [ ]:
## cree 5 tablas
## las sumas que me parecian mas unos ratios..
## otro con lag y delta de dos meses
## promedio de todos los campos
## regr_slope de todos los campos
## per_Rank

##.. ahora tengo que unir esto

In [4]:
%%sql
SELECT * FROM competencia_01_percent_rank 


,numero_de_cliente,foto_mes,percent_rank_mcuentas_saldo,percent_rank_cpayroll_trx,percent_rank_ctrx_quarter,percent_rank_mcaja_ahorro,percent_rank_mprestamos_personales,percent_rank_mtarjeta_visa_consumo,percent_rank_mpasivos_margen,percent_rank_mcomisiones_mantenimiento,percent_rank_cproductos
0,600226432,202102,0.054622,0.000000,0.000000,0.000069,0.000000,0.000000,0.032022,0.728090,0.000000
1,600226432,202101,0.064648,0.000000,0.000000,0.000069,0.000000,0.000000,0.032022,0.728090,0.000000
2,1168447427,202101,0.161096,0.000000,0.049220,0.106370,0.915574,0.000000,0.058809,0.728090,0.000000
3,1168447427,202102,0.157554,0.000000,0.063402,0.149362,0.914701,0.000000,0.076374,0.728090,0.000000
4,985836090,202101,0.396031,0.000000,0.067243,0.404743,0.892866,0.000000,0.242165,0.728090,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
981941,747703842,202103,0.954645,0.459453,0.971773,0.558679,0.866152,0.813846,0.013990,0.014346,0.999952
981942,747703842,202102,0.874922,0.459453,0.971449,0.594720,0.868405,0.915984,0.029428,0.014346,0.999952
981943,316695411,202102,0.550156,0.750350,0.991801,0.471971,0.900591,0.992488,0.151916,0.014346,0.999998
981944,316695411,202104,0.619428,0.459453,0.992751,0.410720,0.845084,0.983428,0.838171,0.014346,0.999998


In [46]:
%%sql 
create or replace table competencia_01_fe as
SELECT * FROM competencia_01_suma_division as suma

LEFT JOIN competencia_01_lag_delta as lag
ON suma.numero_de_cliente = lag.numero_de_cliente 
    AND suma.foto_mes = lag.foto_mes  

LEFT JOIN competencia_01_avg as avg 
ON suma.numero_de_cliente = avg.numero_de_cliente 
    AND suma.foto_mes = avg.foto_mes  
    
LEFT JOIN competencia_01_regr_slope as reg 
ON suma.numero_de_cliente = reg.numero_de_cliente 
    AND suma.foto_mes = reg.foto_mes

LEFT JOIN competencia_01_percent_rank as p_rank
ON suma.numero_de_cliente = p_rank.numero_de_cliente 
    AND suma.foto_mes = p_rank.foto_mes

RuntimeError: (duckdb.duckdb.OutOfMemoryException) Out of Memory Error: could not allocate block of size 2.0 KiB (6.2 GiB/6.2 GiB used)
[SQL: create or replace table competencia_01_fe as
SELECT * FROM competencia_01_suma_division as suma

LEFT JOIN competencia_01_lag_delta as lag
ON suma.numero_de_cliente = lag.numero_de_cliente
    AND suma.foto_mes = lag.foto_mes

LEFT JOIN competencia_01_avg as avg
ON suma.numero_de_cliente = avg.numero_de_cliente
    AND suma.foto_mes = avg.foto_mes

LEFT JOIN competencia_01_regr_slope as reg
ON suma.numero_de_cliente = reg.numero_de_cliente
    AND suma.foto_mes = reg.foto_mes

LEFT JOIN competencia_01_percent_rank as p_rank
ON suma.numero_de_cliente = p_rank.numero_de_cliente
    AND suma.foto_mes = p_rank.foto_mes]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
If you need help solving this issue, send us a message: https://ploomber.io/community


## Como todas no las puedo unir de una voy de a pasos

In [4]:
%%sql
PRAGMA memory_limit='16GB'  -- Ajusta el límite de memoria

,Success


In [3]:
%%sql
CREATE OR REPLACE TABLE competencia_01_intermedia_1 AS
SELECT *
FROM competencia_01_suma_division suma
LEFT JOIN competencia_01_lag_delta lag
ON suma.numero_de_cliente = lag.numero_de_cliente 
   AND suma.foto_mes = lag.foto_mes

,Success


In [2]:
%%sql
CREATE OR REPLACE TABLE competencia_01_intermedia_2 AS
SELECT *
FROM competencia_01_intermedia_1 inter1
LEFT JOIN competencia_01_avg avg
ON inter1.numero_de_cliente = avg.numero_de_cliente 
   AND inter1.foto_mes = avg.foto_mes

,Success


In [5]:
%%sql
CREATE OR REPLACE TABLE competencia_01_intermedia_3 AS
SELECT *
FROM competencia_01_intermedia_2 inter2
LEFT JOIN competencia_01_regr_slope reg
ON inter2.numero_de_cliente = reg.numero_de_cliente 
   AND inter2.foto_mes = reg.foto_mes

,Success


In [57]:
%%sql
select * from competencia_01_intermedia_3
limit 2

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_madelantodolares_slope_3,Visa_fultimo_cierre_slope_3,Visa_mpagado_slope_3,Visa_mpagospesos_slope_3,Visa_mpagosdolares_slope_3,Visa_fechaalta_slope_3,Visa_mconsumototal_slope_3,Visa_cconsumos_slope_3,Visa_cadelantosefectivo_slope_3,Visa_mpagominimo_slope_3
0,476303659,202102,1,0,0,46,25,1600.61,5493.24,1501.83,...,0.0,-7.0,0.0,1130.630,0.0,28.0,0.0,0.0,0.0,46.92
1,476303659,202103,1,0,0,46,26,2092.29,7341.63,1982.46,...,0.0,-1.5,0.0,202.195,0.0,29.5,0.0,0.0,0.0,23.46


In [6]:
%%sql
CREATE OR REPLACE TABLE competencia_01_fe AS
SELECT *
FROM competencia_01_intermedia_3 inter3
LEFT JOIN competencia_01_percent_rank p_rank
ON inter3.numero_de_cliente = p_rank.numero_de_cliente 
   AND inter3.foto_mes = p_rank.foto_mes

,Success


In [2]:
%%sql
select * from competencia_01_fe 
where foto_mes=202104

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,foto_mes_4,percent_rank_mcuentas_saldo,percent_rank_cpayroll_trx,percent_rank_ctrx_quarter,percent_rank_mcaja_ahorro,percent_rank_mprestamos_personales,percent_rank_mtarjeta_visa_consumo,percent_rank_mpasivos_margen,percent_rank_mcomisiones_mantenimiento,percent_rank_cproductos
0,781859013,202104,1,0,0,55,162,616.97,-1242.10,1775.46,...,202104,0.094201,0.000000,0.420242,0.078536,0.801012,0.682662,0.254337,0.787311,0.237888
1,782496982,202104,1,0,0,68,147,490.68,56860.10,2899.17,...,202104,0.511786,0.000000,0.693336,0.776148,0.000000,0.611329,0.674968,0.910120,0.237888
2,782705847,202104,1,0,0,45,134,-266.40,17368.58,1645.05,...,202104,0.506172,0.000000,0.958330,0.719125,0.000000,0.861699,0.680369,0.709605,0.013704
3,783844305,202104,1,0,0,46,160,397.95,23483.22,4358.51,...,202104,0.260327,0.000000,0.437381,0.487950,0.000000,0.772384,0.678506,0.975060,0.237888
4,783936269,202104,1,0,0,38,161,-1710.12,12615.29,697.94,...,202104,0.743695,0.459453,0.648539,0.409951,0.962260,0.812814,0.466084,0.014346,0.237888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164085,489074353,202104,1,0,0,42,176,983.07,10897.54,718.81,...,202104,0.549736,0.459453,0.968843,0.675146,0.000000,0.378736,0.568239,0.014346,0.044147
164086,541811055,202104,1,0,0,62,146,5662.52,18561.77,1345.94,...,202104,0.996843,0.459453,0.889668,0.971039,0.000000,0.535107,0.951058,0.014346,0.775330
164087,1174773260,202104,1,0,0,28,46,-1601.64,-21377.42,79.56,...,202104,0.175969,0.459453,0.271840,0.428888,0.000000,0.614108,0.242550,0.014346,0.237888
164088,591325322,202104,1,0,0,55,202,2110.66,16728.69,1684.26,...,202104,0.267518,0.000000,0.102754,0.160675,0.854628,0.162170,0.092685,0.787311,0.546294


Y a no olvidarse guardar las nueva tabla

In [8]:
%%sql
COPY competencia_01_fe TO '../../../datasets/competencia_01_fe.csv' (FORMAT CSV, HEADER TRUE)


,Success


### imposible de levantar luego

voy a guardar cada una de las tablas intermedias y correr el rf. De cada corrida seleccionare las variables mas importantes y al momento de descargar el df final solo lo hare con esas. 

In [2]:
%%sql
COPY competencia_01_suma_division TO '../../../datasets/competencia_01_suma_division.csv' (FORMAT CSV, HEADER TRUE)

,Success


In [6]:
%%sql
COPY competencia_01_lag_delta TO '../../../datasets/competencia_01_lag_delta.csv' (FORMAT CSV, HEADER TRUE)

,Success


In [7]:
%%sql
COPY competencia_01_avg TO '../../../datasets/competencia_01_avg.csv' (FORMAT CSV, HEADER TRUE)

,Success


In [8]:
%%sql
COPY competencia_01_regr_slope TO '../../../datasets/competencia_01_regr_slope.csv' (FORMAT CSV, HEADER TRUE)

,Success


In [9]:
%%sql
COPY competencia_01_percent_rank TO '../../../datasets/competencia_01_percent_rank.csv' (FORMAT CSV, HEADER TRUE)

,Success


Luego de hacer todo esto, creo que voy a empezar de nuevo, con algo mas claro. espero

In [ ]:
%%sql
select * from competencia_01_fe 
where foto_mes=202104

Me traigo todas las variables importantes que encontre con rf y me bajo solo esas columnas

In [2]:
top_features=['ctrx_quarter', 'mcaja_ahorro', 'mtarjetas_consumo', 'mpasivos_margen', 'mtarjeta_visa_consumo', 'cant_transac_tarjetas', 'msaldototal_total', 'm_payroll_total', 'ctarjeta_visa_transacciones', 'cpayroll_trx', 'mpayroll', 'mautoservicio', 'mpagospesos_total', 'msaldopesos_total', 'mcuentas_saldo', 'm_cpayroll_total', 'ctarjeta_debito_transacciones', 'mcomisiones_mantenimiento', 'mtransferencias_recibidas', 'mpagominimo_total', 'ctarjeta_visa', 'cproductos', 'ccomisiones_mantenimiento', 'cconsumos_total', 'cextraccion_autoservicio', 'mcuenta_corriente', 'mcomisiones', 'ctarjeta_master', 'mfinanciacion_limite_total', 'mtransferencias_emitidas', 'cant_tarjetas', 'ccomisiones_otras', 'mcomisiones_otras', 'Visa_msaldopesos', 'mactivos_margen', 'mextraccion_autoservicio', 'transacciones_por_tarjeta', 'mrentabilidad_annual_no_null', 'mconsumospesos_total', 'fechaalta_total']
top_features1=[ 'lag_1_cliente_antiguedad', 'delta_2_mcuentas_saldo', 'delta_1_mactivos_margen', 'lag_2_cliente_antiguedad', 'lag_2_Visa_fechaalta', 'lag_1_Master_fechaalta', 'delta_2_mpasivos_margen', 'lag_2_mrentabilidad', 'delta_2_mactivos_margen', 'lag_2_Visa_mpagominimo', 'lag_1_Visa_fechaalta', 'lag_1_mpasivos_margen', 'lag_1_mcaja_ahorro', 'delta_2_mrentabilidad', 'lag_2_mcuentas_saldo', 'lag_2_mpasivos_margen', 'lag_2_mcaja_ahorro', 'delta_1_mrentabilidad_annual', 'delta_1_mrentabilidad', 'lag_1_mrentabilidad', 'delta_2_mcomisiones_otras', 'lag_2_mactivos_margen', 'delta_2_mrentabilidad_annual', 'lag_2_Master_fechaalta', 'delta_1_mpasivos_margen', 'delta_2_mcomisiones', 'delta_2_mcaja_ahorro', 'delta_1_mcuentas_saldo', 'lag_2_mcuenta_debitos_automaticos', 'lag_1_mrentabilidad_annual', 'lag_1_Visa_mconsumospesos', 'lag_2_Visa_msaldototal', 'delta_1_mcomisiones', 'delta_1_mcomisiones_otras', 'lag_2_mcomisiones_otras', 'lag_1_Visa_msaldototal', 'lag_2_mrentabilidad_annual']
top_features2=['avg_3_Visa_fechaalta', 'avg_3_chomebanking_transacciones', 'avg_3_mactivos_margen', 'avg_3_cliente_antiguedad', 'avg_3_mrentabilidad', 'avg_3_mrentabilidad_annual', 'avg_3_Visa_mpagominimo', 'avg_3_mcuentas_saldo', 'avg_3_Visa_mpagospesos', 'avg_3_cliente_edad', 'avg_3_mpasivos_margen', 'avg_3_mcomisiones', 'avg_3_mcomisiones_otras', 'avg_3_mcaja_ahorro', 'avg_3_mtransferencias_emitidas', 'avg_3_mtarjeta_visa_consumo', 'avg_3_Master_fechaalta', 'avg_3_Visa_Fvencimiento', 'avg_3_Visa_mconsumototal', 'avg_3_Visa_msaldopesos', 'avg_3_mtransferencias_recibidas', 'avg_3_mcuenta_corriente', 'avg_3_mautoservicio', 'avg_3_ctrx_quarter', 'avg_3_mextraccion_autoservicio', 'avg_3_ccomisiones_otras', 'avg_3_Visa_msaldototal', 'avg_3_mttarjeta_visa_debitos_automaticos', 'avg_3_mpagomiscuentas', 'avg_3_Visa_mconsumospesos', 'avg_3_ctarjeta_debito_transacciones', 'avg_3_mcaja_ahorro_dolares', 'avg_3_ctarjeta_visa_transacciones', 'avg_3_matm', 'avg_3_mpayroll', 'avg_3_mcuenta_debitos_automaticos', 'avg_3_Master_Fvencimiento', 'avg_3_ctransferencias_recibidas']
top_features3=['mcuentas_saldo_slope_3', 'mcomisiones_otras_slope_3', 'mactivos_margen_slope_3', 'mrentabilidad_slope_3', 'mcomisiones_slope_3', 'ctrx_quarter_slope_3', 'mpasivos_margen_slope_3', 'mautoservicio_slope_3', 'mrentabilidad_annual_slope_3', 'mcaja_ahorro_slope_3', 'Visa_mpagominimo_slope_3', 'mtransferencias_recibidas_slope_3', 'chomebanking_transacciones_slope_3', 'Visa_mpagospesos_slope_3', 'Visa_msaldototal_slope_3', 'Visa_mconsumototal_slope_3', 'Visa_mconsumospesos_slope_3', 'Visa_msaldopesos_slope_3', 'mcaja_ahorro_dolares_slope_3', 'mpayroll_slope_3', 'mtarjeta_visa_consumo_slope_3', 'ctarjeta_visa_transacciones_slope_3', 'mcuenta_debitos_automaticos_slope_3', 'mcuenta_corriente_slope_3', 'mttarjeta_visa_debitos_automaticos_slope_3', 'mtransferencias_emitidas_slope_3', 'mpagomiscuentas_slope_3', 'ctransferencias_recibidas_slope_3', 'matm_slope_3', 'ctarjeta_debito_transacciones_slope_3', 'mextraccion_autoservicio_slope_3', 'ctransferencias_emitidas_slope_3', 'Visa_cconsumos_slope_3', 'ccomisiones_otras_slope_3', 'Master_mpagospesos_slope_3', 'mtarjeta_master_consumo_slope_3', 'Master_msaldototal_slope_3', 'catm_trx_slope_3', 'cextraccion_autoservicio_slope_3']
top_features4=['percent_rank_mpasivos_margen', 'percent_rank_mcuentas_saldo', 'percent_rank_mtarjeta_visa_consumo', 'percent_rank_mcaja_ahorro', 'percent_rank_ctrx_quarter', 'percent_rank_mprestamos_personales', 'percent_rank_cproductos', 'percent_rank_mcomisiones_mantenimiento', 'percent_rank_cpayroll_trx','Clase_ternaria']

In [3]:
variables_importantes = top_features + top_features1 + top_features2 + top_features3 + top_features4

In [5]:
# Unir las variables en una cadena separada por comas para la consulta SQL
variables_importantes_str = ",\n    ".join(variables_importantes)

# Crear la consulta SQL con formato adecuado
consulta_sql = f"""
CREATE OR REPLACE TABLE competencia_01_fe_abril AS
SELECT 
    numero_de_cliente,
    foto_mes,
    {variables_importantes_str}
FROM competencia_01_fe
WHERE foto_mes = 202104;
"""

# Mostrar la consulta generada
print(consulta_sql)


CREATE OR REPLACE TABLE competencia_01_fe_abril AS
SELECT 
    numero_de_cliente,
    foto_mes,
    ctrx_quarter,
    mcaja_ahorro,
    mtarjetas_consumo,
    mpasivos_margen,
    mtarjeta_visa_consumo,
    cant_transac_tarjetas,
    msaldototal_total,
    m_payroll_total,
    ctarjeta_visa_transacciones,
    cpayroll_trx,
    mpayroll,
    mautoservicio,
    mpagospesos_total,
    msaldopesos_total,
    mcuentas_saldo,
    m_cpayroll_total,
    ctarjeta_debito_transacciones,
    mcomisiones_mantenimiento,
    mtransferencias_recibidas,
    mpagominimo_total,
    ctarjeta_visa,
    cproductos,
    ccomisiones_mantenimiento,
    cconsumos_total,
    cextraccion_autoservicio,
    mcuenta_corriente,
    mcomisiones,
    ctarjeta_master,
    mfinanciacion_limite_total,
    mtransferencias_emitidas,
    cant_tarjetas,
    ccomisiones_otras,
    mcomisiones_otras,
    Visa_msaldopesos,
    mactivos_margen,
    mextraccion_autoservicio,
    transacciones_por_tarjeta,
    mrentabilidad_annual

In [6]:
%%sql
CREATE OR REPLACE TABLE competencia_01_fe_abril as
select
    numero_de_cliente,
    foto_mes,
    ctrx_quarter,
    mcaja_ahorro,
    mtarjetas_consumo,
    mpasivos_margen,
    mtarjeta_visa_consumo,
    cant_transac_tarjetas,
    msaldototal_total,
    m_payroll_total,
    ctarjeta_visa_transacciones,
    cpayroll_trx,
    mpayroll,
    mautoservicio,
    mpagospesos_total,
    msaldopesos_total,
    mcuentas_saldo,
    m_cpayroll_total,
    ctarjeta_debito_transacciones,
    mcomisiones_mantenimiento,
    mtransferencias_recibidas,
    mpagominimo_total,
    ctarjeta_visa,
    cproductos,
    ccomisiones_mantenimiento,
    cconsumos_total,
    cextraccion_autoservicio,
    mcuenta_corriente,
    mcomisiones,
    ctarjeta_master,
    mfinanciacion_limite_total,
    mtransferencias_emitidas,
    cant_tarjetas,
    ccomisiones_otras,
    mcomisiones_otras,
    Visa_msaldopesos,
    mactivos_margen,
    mextraccion_autoservicio,
    transacciones_por_tarjeta,
    mrentabilidad_annual_no_null,
    mconsumospesos_total,
    fechaalta_total,
    lag_1_cliente_antiguedad,
    delta_2_mcuentas_saldo,
    delta_1_mactivos_margen,
    lag_2_cliente_antiguedad,
    lag_2_Visa_fechaalta,
    lag_1_Master_fechaalta,
    delta_2_mpasivos_margen,
    lag_2_mrentabilidad,
    delta_2_mactivos_margen,
    lag_2_Visa_mpagominimo,
    lag_1_Visa_fechaalta,
    lag_1_mpasivos_margen,
    lag_1_mcaja_ahorro,
    delta_2_mrentabilidad,
    lag_2_mcuentas_saldo,
    lag_2_mpasivos_margen,
    lag_2_mcaja_ahorro,
    delta_1_mrentabilidad_annual,
    delta_1_mrentabilidad,
    lag_1_mrentabilidad,
    delta_2_mcomisiones_otras,
    lag_2_mactivos_margen,
    delta_2_mrentabilidad_annual,
    lag_2_Master_fechaalta,
    delta_1_mpasivos_margen,
    delta_2_mcomisiones,
    delta_2_mcaja_ahorro,
    delta_1_mcuentas_saldo,
    lag_2_mcuenta_debitos_automaticos,
    lag_1_mrentabilidad_annual,
    lag_1_Visa_mconsumospesos,
    lag_2_Visa_msaldototal,
    delta_1_mcomisiones,
    delta_1_mcomisiones_otras,
    lag_2_mcomisiones_otras,
    lag_1_Visa_msaldototal,
    lag_2_mrentabilidad_annual,
    avg_3_Visa_fechaalta,
    avg_3_chomebanking_transacciones,
    avg_3_mactivos_margen,
    avg_3_cliente_antiguedad,
    avg_3_mrentabilidad,
    avg_3_mrentabilidad_annual,
    avg_3_Visa_mpagominimo,
    avg_3_mcuentas_saldo,
    avg_3_Visa_mpagospesos,
    avg_3_cliente_edad,
    avg_3_mpasivos_margen,
    avg_3_mcomisiones,
    avg_3_mcomisiones_otras,
    avg_3_mcaja_ahorro,
    avg_3_mtransferencias_emitidas,
    avg_3_mtarjeta_visa_consumo,
    avg_3_Master_fechaalta,
    avg_3_Visa_Fvencimiento,
    avg_3_Visa_mconsumototal,
    avg_3_Visa_msaldopesos,
    avg_3_mtransferencias_recibidas,
    avg_3_mcuenta_corriente,
    avg_3_mautoservicio,
    avg_3_ctrx_quarter,
    avg_3_mextraccion_autoservicio,
    avg_3_ccomisiones_otras,
    avg_3_Visa_msaldototal,
    avg_3_mttarjeta_visa_debitos_automaticos,
    avg_3_mpagomiscuentas,
    avg_3_Visa_mconsumospesos,
    avg_3_ctarjeta_debito_transacciones,
    avg_3_mcaja_ahorro_dolares,
    avg_3_ctarjeta_visa_transacciones,
    avg_3_matm,
    avg_3_mpayroll,
    avg_3_mcuenta_debitos_automaticos,
    avg_3_Master_Fvencimiento,
    avg_3_ctransferencias_recibidas,
    mcuentas_saldo_slope_3,
    mcomisiones_otras_slope_3,
    mactivos_margen_slope_3,
    mrentabilidad_slope_3,
    mcomisiones_slope_3,
    ctrx_quarter_slope_3,
    mpasivos_margen_slope_3,
    mautoservicio_slope_3,
    mrentabilidad_annual_slope_3,
    mcaja_ahorro_slope_3,
    Visa_mpagominimo_slope_3,
    mtransferencias_recibidas_slope_3,
    chomebanking_transacciones_slope_3,
    Visa_mpagospesos_slope_3,
    Visa_msaldototal_slope_3,
    Visa_mconsumototal_slope_3,
    Visa_mconsumospesos_slope_3,
    Visa_msaldopesos_slope_3,
    mcaja_ahorro_dolares_slope_3,
    mpayroll_slope_3,
    mtarjeta_visa_consumo_slope_3,
    ctarjeta_visa_transacciones_slope_3,
    mcuenta_debitos_automaticos_slope_3,
    mcuenta_corriente_slope_3,
    mttarjeta_visa_debitos_automaticos_slope_3,
    mtransferencias_emitidas_slope_3,
    mpagomiscuentas_slope_3,
    ctransferencias_recibidas_slope_3,
    matm_slope_3,
    ctarjeta_debito_transacciones_slope_3,
    mextraccion_autoservicio_slope_3,
    ctransferencias_emitidas_slope_3,
    Visa_cconsumos_slope_3,
    ccomisiones_otras_slope_3,
    Master_mpagospesos_slope_3,
    mtarjeta_master_consumo_slope_3,
    Master_msaldototal_slope_3,
    catm_trx_slope_3,
    cextraccion_autoservicio_slope_3,
    percent_rank_mpasivos_margen,
    percent_rank_mcuentas_saldo,
    percent_rank_mtarjeta_visa_consumo,
    percent_rank_mcaja_ahorro,
    percent_rank_ctrx_quarter,
    percent_rank_mprestamos_personales,
    percent_rank_cproductos,
    percent_rank_mcomisiones_mantenimiento,
    percent_rank_cpayroll_trx,
    Clase_ternaria
FROM competencia_01_fe
WHERE foto_mes = 202104

,Success


In [7]:
%%sql
select * from competencia_01_fe_abril

,numero_de_cliente,foto_mes,ctrx_quarter,mcaja_ahorro,mtarjetas_consumo,mpasivos_margen,mtarjeta_visa_consumo,cant_transac_tarjetas,msaldototal_total,m_payroll_total,...,percent_rank_mpasivos_margen,percent_rank_mcuentas_saldo,percent_rank_mtarjeta_visa_consumo,percent_rank_mcaja_ahorro,percent_rank_ctrx_quarter,percent_rank_mprestamos_personales,percent_rank_cproductos,percent_rank_mcomisiones_mantenimiento,percent_rank_cpayroll_trx,clase_ternaria
0,781859013,202104,91,22.82,35030.11,245.85,33347.65,26,30623.72,0.00,...,0.254337,0.094201,0.682662,0.078536,0.420242,0.801012,0.237888,0.787311,0.000000,CONTINUA
1,782496982,202104,144,80882.65,29211.49,1200.40,26661.23,31,26231.23,0.00,...,0.674968,0.511786,0.611329,0.776148,0.693336,0.000000,0.237888,0.910120,0.000000,CONTINUA
2,782705847,202104,283,61426.96,64194.57,1223.69,64194.57,25,56394.35,0.00,...,0.680369,0.506172,0.861699,0.719125,0.958330,0.000000,0.013704,0.709605,0.000000,CONTINUA
3,783844305,202104,94,22959.22,44909.43,1215.60,44909.43,10,40693.96,0.00,...,0.678506,0.260327,0.772384,0.487950,0.437381,0.000000,0.237888,0.975060,0.000000,CONTINUA
4,783936269,202104,134,16057.77,60678.69,599.51,52201.59,17,53615.49,160773.72,...,0.466084,0.743695,0.812814,0.409951,0.648539,0.962260,0.237888,0.014346,0.459453,CONTINUA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164085,489074353,202104,306,50479.28,11734.28,839.28,11734.28,8,9961.75,90206.52,...,0.568239,0.549736,0.378736,0.675146,0.968843,0.000000,0.044147,0.014346,0.459453,CONTINUA
164086,541811055,202104,213,465556.76,20911.95,6176.69,20911.95,8,18095.17,145645.54,...,0.951058,0.996843,0.535107,0.971039,0.889668,0.000000,0.775330,0.014346,0.459453,CONTINUA
164087,1174773260,202104,64,17582.49,39695.17,228.94,26893.08,16,37488.04,76015.09,...,0.242550,0.175969,0.614108,0.428888,0.271840,0.000000,0.237888,0.014346,0.459453,CONTINUA
164088,591325322,202104,28,1862.79,591.62,33.54,591.62,1,900.83,0.00,...,0.092685,0.267518,0.162170,0.160675,0.102754,0.854628,0.546294,0.787311,0.000000,CONTINUA


In [8]:
%%sql
select * from competencia_01
where foto_mes=202104

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria
0,249619949,202104,1,0,0,45,71,-4296.05,-26240.03,3213.29,...,2.0,0.00,-92740.78,0.00,1723.0,4767.54,3.0,0.0,7542.39,CONTINUA
1,250480925,202104,1,0,0,41,193,2854.65,23269.05,1707.97,...,2.0,0.00,-56376.30,0.00,2997.0,2767.35,8.0,0.0,0.00,CONTINUA
2,250888364,202104,1,0,0,58,323,9988.48,208327.05,5586.36,...,9.0,0.00,-32880.77,20.07,8698.0,23000.50,16.0,0.0,4105.50,CONTINUA
3,250892233,202104,1,0,0,69,323,2013.52,-4137.06,192.24,...,2.0,0.00,NaN,NaN,253.0,NaN,NaN,NaN,0.00,CONTINUA
4,251114514,202104,1,0,0,54,291,5070.43,46194.44,2348.06,...,2.0,0.00,-10498.35,0.00,8839.0,7495.65,2.0,0.0,7518.93,CONTINUA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164085,585056653,202104,1,0,0,45,177,7932.89,64280.55,3674.19,...,2.0,0.00,-9384.00,0.00,5059.0,3650.70,4.0,0.0,5325.42,CONTINUA
164086,585254433,202104,1,0,0,43,203,5003.75,253880.44,4118.73,...,2.0,0.00,-30928.63,0.00,4323.0,15983.32,13.0,0.0,2709.63,CONTINUA
164087,585418447,202104,1,0,0,60,193,1097.85,-7534.15,3442.45,...,2.0,11730.00,-35190.00,0.00,5871.0,30396.05,37.0,0.0,24163.80,CONTINUA
164088,585750413,202104,1,0,0,40,166,688.39,18021.83,378.11,...,2.0,0.00,-23460.00,0.00,4945.0,7175.00,6.0,0.0,20926.32,CONTINUA


In [9]:
%%sql
SELECT numero_de_cliente
FROM competencia_01_fe_abril
WHERE numero_de_cliente NOT IN (SELECT numero_de_cliente FROM competencia_01 WHERE foto_mes = 202104)


,numero_de_cliente


In [11]:
%%sql
SELECT numero_de_cliente
FROM competencia_01
WHERE foto_mes = 202104
AND numero_de_cliente NOT IN (SELECT numero_de_cliente FROM competencia_01_fe_abril)

,numero_de_cliente


## me aseguro que tengo los mismos clientes para el mes de abril en la tabla original como en la tabla final

In [10]:

%%sql
COPY competencia_01_fe_abril TO '../../../datasets/competencia_01_fe_abril.csv' (FORMAT CSV, HEADER TRUE)

,Success
